# Dashbord Project Part 1: Data Analisys

In [1]:
#libraries
#----------------------------------------------
import pandas as pd
from pylab import * 
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt #to plot graphs
import matplotlib.patches as mpatches #to customize legend
import seaborn as sns #to plot graphs
import math 
import re #to find integers & floats in a string 
from statsmodels.graphics.mosaicplot import mosaic #for mosaic plot
from collections import Counter #to count values in a dataframe or array
from datetime import date, timedelta, datetime #for the date
import datetime as dt
import pylab 
import scipy.stats as stats
import statsmodels
from colorama import Fore, Back, Style #for color in print

## data extraction and cleaning 

### Load file

In [2]:
#reading the excel file
#----------------------------------------------
#path= 'C:\\Users\\coxi\\Desktop\\PROJECT_INTERSHIP\\Meetings&marks_anonyomized_v2.xlsx' #caution to double the '\'
path='data/' #if the file is in the current directory
completeDashbord = pd.read_excel(path+'Meetings&marks_anonyomized_v2.xlsx')
completeDashbord.head()

Index  approved                                           comments  \
0    0.0       0.0                                     This is a test   
1    1.0       1.0  Watch out for peer reviews journals looking fo...   
2    2.0       NaN  Discussed general outline of paper i.e. resear...   
3    3.0       NaN  Ed emailed the progress report below in advanc...   
4    4.0       1.0                                                NaN   

        date                                       deliverables duration  \
0 2017-11-22                                                NaN      NaN   
1 2017-12-07                                                NaN      NaN   
2 2018-02-20  Identify suitable collection of malicious VB s...      NaN   
3 2018-07-13                                                NaN      NaN   
4 2018-01-23                                                NaN      NaN   

  module  Year      progress      type Marks Student Supervisor  
0    mcm  2018  satisfactory  approval    66       4         17  
1    mcm  2018  satisfactory  approval    66       4         17  
2    mcm  2018  satisfactory   meeting    66       4         17  
3    mcm  2018  satisfactory   meeting    66       4         17  
4    mcm  2018  satisfactory  approval    58      14         41

In [3]:
#remove unused columns
#----------------------------------------------
#remove the "Index" column 
dashbord = completeDashbord.drop('Index', inplace=False, axis=1)
#remove the "comments" and "deliverables" columns for now
dashbord.drop('comments', inplace=True, axis=1)
dashbord.drop('deliverables', inplace=True, axis=1)
dashbord.head()

approved       date duration module  Year      progress      type Marks  \
0       0.0 2017-11-22      NaN    mcm  2018  satisfactory  approval    66   
1       1.0 2017-12-07      NaN    mcm  2018  satisfactory  approval    66   
2       NaN 2018-02-20      NaN    mcm  2018  satisfactory   meeting    66   
3       NaN 2018-07-13      NaN    mcm  2018  satisfactory   meeting    66   
4       1.0 2018-01-23      NaN    mcm  2018  satisfactory  approval    58   

  Student Supervisor  
0       4         17  
1       4         17  
2       4         17  
3       4         17  
4      14         41

### Clean duration

The duration column needs to be cleaned :

In [4]:
def clean_duration(duration):
    """ 
    This function harmonizes the duration. 
    
    --> Duration is not always entered correctly. One can find: 
    10,nan,10min,10 min,/,0.5h,1hour,1 hour,1 H,1Hour,h1.5,h 0.5,H 1,various,1+5,5x2,13:30-14:30... 
    (cases considered here)
    --> If there is an unknown character (see in the code which characters are considered correct),
    the function will assign np.nan to the string
    --> If there is an 'h' or 'H' in the string, the function will detect that the time is in hours and 
    convert it in minutes, otherwise it is considered as minutes by default. 
    --> If there are strings such as : "1 hour 15 minutes" the function will only retain 60 minutes

    -  input :     duration:  list of mixed type string and integer
    - output :     duration: list of type integer cleaned with time in minutes
               outliers_idx: list of indexes of outliers values  
    """
    result = np.zeros(len(duration))
    outliers_idx = []
    i = 0
    for s in duration:
        s = str(s) #convert everything to string (to avoid problems)
        #extraction of any unknown caracter from the string i.e. find everything exept:
        #characters minutehoursMINUTESHOURS, integer, floats, blank space, . 
        ru = re.findall("(?:(?![minutehoursMINUTEHOURs\f\d.,\s*]).)*",s)
        unknown = 0

        for elem in ru: #check if there is an unknown caracter in the string
            if len(elem) != 0: 
                unknown = 1
                if s != "nan": #if the unknown caracter is not part of the "nan" string
                    outliers_idx.append(i) #retrieve the index of the unknown caracter
                break  
        
        if unknown == 1: #if there is an unknown caracter in the string, put the string to nan 
            result[i] = np.nan
        else: #if not, extraction of the time from the string 
            #extraction of integers regardless of their position from the string
            rr = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", s)
                
            if len(rr) != 0: #if there is a time in the string
                rr = float(rr[0]) #transform string: "1.3" to float: 1.3
                #if there is an h or H in the string it means that the time is in hours and not in minutes
                rh = re.findall("[hH]+",s)
                #in that case convert hours to minutes
                if len(rh) != 0:
                    rr = rr*60
            result[i] = rr  
        i += 1
    return result,outliers_idx

#documentation:
#https://docs.python.org/3/library/re.html
#https://stackoverflow.com/questions/4289331/how-to-extract-numbers-from-a-string-in-python/4289415#4289415
#https://dl.icewarp.com/online_help/203030104.htm

In [5]:
#test of the function clean_duration with various cases
#----------------------------------------------
ss = [10,np.nan,"10","10min","/","20 min","140","0.5h","1hour","1H","1 hour", "1 Hour","h1.5","H 0.5","1+5+7+7","Andrea","5x2", "13:30-14:30"]
print("--- original string: --- \n",ss)
print("--- cleaned string: --- \n", clean_duration(ss))
### Q: Do I have to consider the empty case : "" ? 

--- original string: --- 
 [10, nan, '10', '10min', '/', '20 min', '140', '0.5h', '1hour', '1H', '1 hour', '1 Hour', 'h1.5', 'H 0.5', '1+5+7+7', 'Andrea', '5x2', '13:30-14:30']
--- cleaned string: --- 
 (array([ 10.,  nan,  10.,  10.,  nan,  20., 140.,  30.,  60.,  60.,  60.,
        60.,  90.,  30.,  nan,  nan,  nan,  nan]), [4, 14, 15, 16, 17])


In [6]:
#clean the duration column
#------------------------------------------------
dur = dashbord["duration"].values.tolist() #convert duration column into a list
dur,outliers_idx = clean_duration(dur) #clean the duration column
print("Outliers indexes and values in the duration column (ie use .iloc[]):")
print(dashbord["duration"].iloc[outliers_idx])

Outliers indexes and values in the duration column (ie use .iloc[]):
1571                            /
1754                            /
1811                            /
1982                            /
2437                            /
2450                            /
2556                            /
2978                            /
3337                    5x2 hours
3410                14.30 - 15:30
5175                      various
5934                      various
9823     25 minutes approximately
10741                     1+5+7+7
10776                       1 day
Name: duration, dtype: object


In [7]:
dashbord["duration"]= pd.DataFrame(dur) #update column duration
#durationIndex = np.where(dashbord["duration"].isna() == False) #indexes where there is a duration

In [8]:
#number of meetings with a duration (having removed approvals and ethics)
#-----------------------------------------------------------------
idMeeting = np.where(dashbord["type"]=='meeting')
nbMeeting = len(dashbord["type"].iloc[idMeeting]) 
nbMeetingDur = len(dashbord["duration"].iloc[idMeeting].iloc[np.where(dashbord["duration"].iloc[idMeeting].isna()==False)])
nbMeetingNoDur = len(dashbord["duration"].iloc[idMeeting].iloc[np.where(dashbord["duration"].iloc[idMeeting].isna()==True)])
print("Total number of meetings: ",nbMeeting)
print("Number of meetings with a  duration: ",nbMeetingDur)
print("Number of meetings with no duration: ",nbMeetingNoDur)
print("Proportion of meetings with a duration: {}%".format(round(nbMeetingDur/nbMeeting,3)*100))

Total number of meetings:  8575
Number of meetings with a  duration:  3086
Number of meetings with no duration:  5489
Proportion of meetings with a duration: 36.0%


### Assign the correct type to variables

In [9]:
#assign the correct type to variables
#----------------------------------------------
### categorical variables
dashbord["approved"]=pd.Categorical(dashbord["approved"],ordered=False)
dashbord["module"]=pd.Categorical(dashbord["module"],ordered=False)
dashbord["type"]=pd.Categorical(dashbord["type"],ordered=False)
dashbord["progress"]=pd.Categorical(dashbord["progress"],ordered=False)

### time variables
dashbord["date"]=pd.to_datetime(dashbord["date"])

In [10]:
### quantitative variables
#---- Marks
nanMarksIndex = np.where(dashbord["Marks"] == ' ') #indexes where there is no mark
dashbord["Marks"].iloc[nanMarksIndex] = np.nan #where there is no mark put value NaN
dashbord["Marks"]= dashbord["Marks"].astype(float) #convert to float 
#dashbord["Marks"] contains np.nan (of type float) and integers 
#we convert to float the whole dashbord["Marks"] to be able to analyse it later.
#If there is no conversion, dashbord["Marks"] will be of type object (mixed between integer and float)
#while pd.NA is of type integer, it is experimental and can have unwanted outcome. 

#---- Student
nanStudentIndex = np.where(dashbord["Student"] == ' ') #indexes where there is no Student ID
dashbord["Student"].iloc[nanStudentIndex] = np.nan #where there is no Student ID put value NaN
dashbord["Student"]= dashbord["Student"].astype(float) #convert to float 

#---- Supervisor
nanSupervisorIndex = np.where(dashbord["Supervisor"] == ' ') #indexes where there is no Supervisor iD
dashbord["Supervisor"].iloc[nanSupervisorIndex] = np.nan #where there is no Supervisor ID put value NaN
dashbord["Supervisor"]= dashbord["Supervisor"].astype(float) #convert to float 

dashbord.dtypes
#### a warning appears because of cascade assignments with panda dataframe ####

C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\119349564.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dashbord["Marks"].iloc[nanMarksIndex] = np.nan #where there is no mark put value NaN
C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\119349564.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dashbord["Student"].iloc[nanStudentIndex] = np.nan #where there is no Student ID put value NaN
C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\119349564.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

approved            category
date          datetime64[ns]
duration             float64
module              category
Year                   int64
progress            category
type                category
Marks                float64
Student              float64
Supervisor           float64
dtype: object

__Variables:__
- **date**: time (can be transformed to categorical)
- **duration**: <font color = "red"> quantitative continuous </font>
- **module**: <font color = "blue"> categorical </font>(5 categories: ca326, ca400, ca472, mcm, pnu)
- **Year**: <font color = "green">quantitative discrete </font> 
- **progress**: <font color = "blue"> categorical </font> (3 categories: satisfactory, moderate, unsatisfactory)
- **type**: <font color = "blue"> categorical </font> (3 categories: approval, meeting, ethics)
- **Marks**: <font color = "green">quantitative discrete </font>  (or can be seen as continuous)
- **Student**: <font color = "green">quantitative discrete </font>
- **Supervisor** <font color = "green">quantitative discrete   </font>

The type is "object" when there are mixed integer and NaN (of type float) values

In [11]:
dashbord.head()

approved       date  duration module  Year      progress      type  Marks  \
0      0.0 2017-11-22       NaN    mcm  2018  satisfactory  approval   66.0   
1      1.0 2017-12-07       NaN    mcm  2018  satisfactory  approval   66.0   
2      NaN 2018-02-20       NaN    mcm  2018  satisfactory   meeting   66.0   
3      NaN 2018-07-13       NaN    mcm  2018  satisfactory   meeting   66.0   
4      1.0 2018-01-23       NaN    mcm  2018  satisfactory  approval   58.0   

   Student  Supervisor  
0      4.0        17.0  
1      4.0        17.0  
2      4.0        17.0  
3      4.0        17.0  
4     14.0        41.0

In [12]:
#export to cvs to be able to work with R
#-------------------------------------------
dashbord.to_csv('result/dashbord.csv')

In [13]:
#pickle dashbord + text = dashbord2 for later use
#-------------------------------------------
dashbord2 = dashbord.copy()
dashbord2["comments"] = completeDashbord["comments"].copy()
dashbord2["deliverables"] = completeDashbord["deliverables"].copy()
dashbord2.to_pickle("result/dashbord2.pkl")

### First simple description

In [14]:
print("number of rows in the dataset: ", completeDashbord.shape[0])
print("number of columns in the dataset: ", completeDashbord.shape[1])

number of rows in the dataset:  11789
number of columns in the dataset:  13


In [15]:
print("---------------------------------------\n approved:")
print(dashbord["approved"].describe())
print("---------------------------------------\n date:")
print(dashbord["date"].describe(datetime_is_numeric=True))
print("---------------------------------------\n duration(min):")
print(dashbord["duration"].describe())
print("---------------------------------------\n module:")
print(dashbord["module"].describe())
print("---------------------------------------\n Year:")
print(dashbord["Year"].describe())
print("--------------------------------------\n progress:")
print(dashbord["progress"].describe())
print("---------------------------------------\n type:")
print(dashbord["type"].describe())
print("---------------------------------------\n Marks:")
print(dashbord["Marks"].describe())
print("---------------------------------------\n Student:")
print("count: ", dashbord["Student"].count())
print("min: ", dashbord["Student"].min())
print("max: ", dashbord["Student"].max())
print("---------------------------------------\n Supervisor:")
print("count: ", dashbord["Supervisor"].count())
print("min: ", dashbord["Supervisor"].min())
print("max: ", dashbord["Supervisor"].max())

---------------------------------------
 approved:
count     3214.0
unique       2.0
top          1.0
freq      2389.0
Name: approved, dtype: float64
---------------------------------------
 date:
count                            11789
mean     2020-05-03 13:44:44.502502144
min                2017-10-08 00:00:00
25%                2019-04-12 00:00:00
50%                2020-04-06 00:00:00
75%                2021-03-18 00:00:00
max                2022-02-15 00:00:00
Name: date, dtype: object
---------------------------------------
 duration(min):
count    3086.000000
mean       25.419961
std        17.721817
min         0.000000
25%        15.000000
50%        20.000000
75%        30.000000
max       480.000000
Name: duration, dtype: float64
---------------------------------------
 module:
count     11789
unique        5
top         mcm
freq       4631
Name: module, dtype: object
---------------------------------------
 Year:
count    11789.000000
mean      2020.211383
std          1.08

The maximum is 480 minutes which corresponds to 8 hours. 
Taking the original dashbord without modifying the duration: \
*np.where(dashbord["duration"] == 480.0) \
dashbord["duration"].iloc[2212]*\
gives 8 hours...


For categorical variables: \
unique : number of categories \
top : top value by alphabetical order 

__Remark:__ There is Students who have no supervisor and supervisor who have no students

## Functions to compute dataset insights 

### retrieveInfo()

In [16]:
def retrieveInfo():
    #---Extracts the indices of students who received a grade. 
    #---As 1 student corresponds to several indices, the first occurrence is taken. 
    #stuIndex = Indices (row labels in the dataframe i.e. use .loc[stuIndex]) of students who received a grade
    stuIndex = dashbord["Student"].drop_duplicates().index 
    #find the index of the NaN value in the dataframe and remove it:
    nanStuIndex = np.where(dashbord["Student"].drop_duplicates().isna() == True) 
    stuIndex = np.delete(stuIndex,nanStuIndex) #remove the NaN value from StuIndex
    nbStuG = len(stuIndex) #number of Students who received a grade in the dataframe

    #among students who received a grade...
    #...find the ID of supervisors who are not NaN values and put these ID in variable supID:
    supIDG = dashbord["Supervisor"].loc[stuIndex].drop_duplicates()
    supIDG = delete(supIDG.values,np.where(supIDG.isna() == True))

    return stuIndex,nbStuG,supIDG
#documentation:
#https://www.tutorialspoint.com/python-pandas-return-unique-values-in-the-index#

<font color = "red"> There can be several supervisors for one student and several students for one supervisor

### computeProgress(...)

In [17]:
def computeProgress(name = "Supervisor",stacked = False,scale = True):
    """
    This function computes statistics for Supervisors or Students.
    ---> input: 
        - name (string): "Student" or "Supervisor" to have students or supervisors statistics
        - stacked (boolean): if True the function gives a dataframe suited for a stacked histogram
        - scale (boolean): if True the function scales the number of satisfacory, moderate and 
                           unsatisfactory so that it sums to 1
    ---> output: dfProg: dataframe containing students or supervisors statistics. In the dataframe the columns: 
        - unsatisfactory : proportion of unsatisfactory progress (among non NaN progress)
        - moderate : proportion of moderate progress (among non NaN progress)
        - satisfactory : proportion of satisfactory progress (among non NaN progress)
        - Mean_mark : mean mark for each supervisor
        - Median_mark : median mark for each supervisor ####
        - nb_students : number of students for each supervisor
        - %FilledDuration : percentage of meetings filled with a duration for each supervisor
    """
    if name == "Supervisor":
        newDash = dashbord[["Marks","Student","Supervisor"]].drop_duplicates()#conserve only relevant information for supervisor statistics
    elif name == "Student":
        newDash = dashbord[["Marks","Student","module","Year"]].drop_duplicates()#conserve only relevant information for student statistics
        newDash.dropna(axis=0,inplace=True) #remove rows with NaN values
        
    newDash = newDash.sort_values(by=[name]) #sort by supervisor/student IDs
    supList = newDash[name].drop_duplicates()#list of all supervisor/student IDs
    #supList.dropna(inplace = True) 
    supList = np.delete(supList.values,np.where(supList.isna() == True)) #delete NaN value from supList 
    meanMark = np.zeros(len(supList)) #mean Mark for each supervisor (in the same order as supList)
    medianMark = np.zeros(len(supList)) #median Mark for each supervisor (in the same order as supList) ####
    nbStudperSup = np.zeros(len(supList)) #number of students per Supervisor
    nbMeetperSup = np.zeros(len(supList)) #number of meetings per Supervisor
    perFilledDur = np.zeros(len(supList)) #percentage of meetings with a duration per Supervisor

    Prog = np.zeros((3,len(supList))) #tab containing the progress for each supervisor/student
    #columns : supervisors (in the same order as supList)
    #row 0: number of 'satisfactory' progress
    #row 1: number of 'moderate' progress
    #row 2: number of 'unsatisfactory' progress
    
    i=0
    for s in supList:

        ##### 1.Computation of the number or proportion of satisfactory/moderate/undatisfactory for each supervisor
        idx = np.where(dashbord[name]==s) #index (raw index no row name) of the location of supervisor s in 'dashbord'

        #--- for 'satisfactory'
        idx_satis = np.where(dashbord["progress"].iloc[idx]=="satisfactory") #index of 'satisfactory' progress  
        nb_satis = len(dashbord["progress"].iloc[idx].iloc[idx_satis]) #number of 'satisfactory' for supervisor s
        Prog[0,i] = nb_satis
        #--- for 'moderate'
        idx_moder = np.where(dashbord["progress"].iloc[idx]=="moderate") #index of 'moderate' progress  
        nb_moder = len(dashbord["progress"].iloc[idx].iloc[idx_moder]) #number of 'moderate' for supervisor s
        Prog[1,i] = nb_moder
        #--- for 'unsatisfactory'
        idx_unsat = np.where(dashbord["progress"].iloc[idx]=="unsatisfactory") #index of 'unsatisfactory' progress  
        nb_unsat = len(dashbord["progress"].iloc[idx].iloc[idx_unsat]) #number of 'unsatisfactory' for supervisor s
        Prog[2,i] = nb_unsat

        if scale == True: #if scale = True, the output will a proportion
            Sum = nb_satis+nb_moder+nb_unsat #so we divide by the total number of 'progress' given by the supervisor         
        else: #if scale = False, the output will be the actual value
            Sum = 1 
        
        if Sum != 0: 
            Prog[:,i] = Prog[:,i]/(Sum)
        else: #if there is no progress, we but the progress at 0
            Prog[:,i] = 0      

        if name == "Supervisor": #if we are looking for Supervisors statistics
        ##### 2.Computation of the mean and median mark given by each supervisor 
            allMarks = newDash["Marks"].iloc[np.where(newDash["Supervisor"]==s)]
            #if the marks are not all NaN values, compute mean and median
            if not(np.all(allMarks.isna())): 
                meanMark[i] = np.nanmean(newDash["Marks"].iloc[np.where(newDash["Supervisor"]==s)]) #mean skip NaN values unless there is only Nan
                medianMark[i] = np.nanmedian(newDash["Marks"].iloc[np.where(newDash["Supervisor"]==s)]) #median skip NaN values unless there is only Nan
            #else, put mean and median to NaN
            else:
                meanMark[i] = np.nan
                medianMark[i] = np.nan   
        ##### 3.Computation of the number of students for each supervisor 
            students = newDash["Student"].iloc[np.where(newDash["Supervisor"]==s)].values #student IDs associated with supervisor s
            students = np.delete(students,np.where(np.isnan(students)==True)) #delete student value equals to NaN 
            nbStudperSup[i] = len(students) #number of students for supervisor s 
        ##### 4.Computation of the number of meetings (no ethics nor approvals) for each supervisor
            meetings = dashbord["type"].iloc[idx] #all types (meetings/approvals/ethics) for supervisor s
            idx_m = np.where(meetings=="meeting") #indexes of meetings among the 'dashbord' restricted to supervisor ID = s
            nbMeetperSup[i] = meetings.iloc[idx_m].count() #number of meetings for supervisor s
            #nbMeetperSup[i] = meetings.iloc[np.where(meetings=="meeting")].count() #number of meetings for supervisor s
        ##### 5.Computation of the percentage of meetings with a duration for each supervisor
            perFilledDur[i] = dashbord["duration"].iloc[idx].iloc[idx_m].count() #number of meetings with a duration for the supervisor ID = s
            perFilledDur[i] = round((perFilledDur[i]/nbMeetperSup[i])*100,1) #compute a percentage round to one decimal point          
        i+=1

    #Data formatting
    #-------------------------------------------------
    dfProg = pd.DataFrame(Prog.T) #transformation to dataframe to plot graph with seaborn
    dfProg = dfProg.rename(columns={0: "satisfactory", 1: "moderate", 2: "unsatisfactory"}) #rename columns       
    dfProg[name+'_ID'] = supList.astype(int) #add column with supervisor IDs
    dfProg = dfProg[[name+'_ID',"unsatisfactory", "moderate", "satisfactory"]] #reoganise columns' position

    if name == "Supervisor": #add statistics for Supervisors
        dfProg["Mean_mark"] = meanMark #add mean mark per supervisor
        dfProg["Median_mark"] = medianMark #add median mark per supervisor
        dfProg["nb_students"] = nbStudperSup.astype(int) #add number of students per supervisor
        dfProg["nb_meetings"] = nbMeetperSup.astype(int) #add number of meetings per supervisor
        dfProg["%FilledDuration"] = perFilledDur #add number of meetings per supervisor
        
    if name == "Student": #add statistics for Students
        dfProg["Marks"] = newDash["Marks"].values.astype(int) #add students' marks
        dfProg["module"] = newDash["module"].values #add students' module
        dfProg["Year"] = newDash["Year"].values #add Year
        
    #Data formatting for stacked histogram
    #-------------------------------------------------
    if stacked == True:
        dfProg.index = dfProg[name+'_ID']
        dfProg.drop(name+'_ID', inplace=True, axis=1)
        
    return(dfProg)

### computeWorstProgress()

In [18]:
def computeWorstProgress():
    """
    This function calculates the worst progress of the students.
    - If a student has at least 1 unsatisfactory progress, they fall into the unsatisfactory box
    - If a student has 0 unsatisfactory but at least 1 moderate progress, they fall into the moderate box
    - If a student has 0 unsatisfcatory and 0 moderate progress, they fall into the satisfactory box
    ---> output: dfProgStu_n (dataframe): the same dataframe as the one given by the function
    "computeProgress(name="Student",scale = False)" but 3 columns are added:
    - column "unsatisfactory_box": 1 if the student fell into the unsatisfactory box, 0 otherwise*
    - column "moderae_box"       : 1 if the student fell into the moderate box, 0 otherwise
    - column "satisfactory_box"  : 1 if the student fell into the satisfactory box, 0 otherwise    
    
    """  
    #-----use of the function computeProgress
    dfProgStu_n = computeProgress(name="Student",scale = False)
    #-----initialisation of the dataframe with 3 empty columns corresponding to
    #     "unsatisfactory_box","moderate_box" and "satisfactory_box"
    dfProgStu_n["unsatisfactory_box"] = np.zeros(len(dfProgStu_n)) 
    dfProgStu_n["moderate_box"] = np.zeros(len(dfProgStu_n)) 
    dfProgStu_n["satisfactory_box"] = np.zeros(len(dfProgStu_n)) 
    
    #-----sets the value to 1 for column "unsatisfactory_box" if the students fall into the unsatisfactory box
    Ifall_u = np.where(dfProgStu_n["unsatisfactory"]>=1) #indexes where there is at least 1 unsatisfactory progress
    dfProgStu_n["unsatisfactory_box"].iloc[Ifall_u] = 1 #put value 1 for indexes Ifall_u
    #-----sets the value to 1 for column "moderate_box" if the students fall into the moderate box
    cond1 = (dfProgStu_n["moderate"]>=1).values
    cond2 = (dfProgStu_n["unsatisfactory_box"] != 1).values
    Ifall_m = np.where(cond1 & cond2) #indexes where there is no unsatisfactory progress but at least 1 moderate progress 
    dfProgStu_n["moderate_box"].iloc[Ifall_m] = 1 #put value 1 for indexes Ifall_m
    #-----sets the value to 1 for column "satisfactory_box" if the students fall into the satisfactory box
    cond1 = (dfProgStu_n["unsatisfactory_box"]!=1).values
    cond2 = (dfProgStu_n["moderate_box"]!=1).values
    Ifall_s = np.where(cond1 & cond2) #indexes where there is only satisfactory progress 
    dfProgStu_n["satisfactory_box"].iloc[Ifall_s] = 1 #put value 1 for indexes Ifall_s
    #----- create new dataframes: 
    #create new dataframe with only students who fell into the unsatisfactory box
    dfUnsat = dfProgStu_n.iloc[np.where(dfProgStu_n["unsatisfactory_box"]==1)].drop(labels = ["moderate_box","satisfactory_box"],axis=1).copy()
    #create new dataframe with only students who fell into the moderate box
    dfModer = dfProgStu_n.iloc[np.where(dfProgStu_n["moderate_box"]==1)].drop(labels = ["unsatisfactory_box","satisfactory_box"],axis=1).copy()
    #create new dataframe with only students who fell into the satisfactory box
    dfSatis = dfProgStu_n.iloc[np.where(dfProgStu_n["satisfactory_box"]==1)].drop(labels = ["unsatisfactory_box","moderate_box"],axis=1).copy()
    return (dfUnsat,dfModer,dfSatis)

### computeModule(...)

In [19]:
def computeModule(df,dashbord):
    """
    ---> input: 
    - dfProgSup = computeProgress()
    - dashbord: the original student dashbord with duration cleaned
    ---> output:
    - dfProgSup: adds to the existing dataframe dfProgSup as many columns as there are modules. 
                 Each column corresponds to a module and contains the values 0 or 1. 
                 A value of 1 indicates that the corresponding supervisor teaches the module 
                 and a value of 0 indicates that the corresponding supervisor does not teach the module. 
    """
    dfProgSup = df.copy()
    modList = dashbord["module"].drop_duplicates().values
    supList = np.sort(dashbord["Supervisor"].dropna(axis=0).drop_duplicates().values).astype(int) #list of supervisor IDs
    teachMod = np.zeros((len(dfProgSup),len(modList))) #table containing the modules taught by each supervisor
    #rows : supervisor IDs
    #column j : 1 if supervisor teaches the module modList[j], 0 otherwise

    i=0
    for s in supList:
        idx = np.where(dashbord["Supervisor"]==s) #index of the location of supervisor s in 'dashbord'
        s_mod = dashbord["module"].iloc[idx].drop_duplicates().values #modules taught by supervisor s
        for j in range(len(modList)):
            if modList[j] in s_mod: #if the supervisor s teaches the module modList[j]...
                teachMod[i,j] = 1 #... the value of the column j of teachMod becomes 1
        i+=1   

        for j in range(len(modList)):
            dfProgSup[modList[j]] = teachMod[:,j].astype(int)
    return(dfProgSup)

### computeMostCommonProgress(...)

In [20]:
def computeMostCommonProgress(stuIndex):
    """
    - For each student returns the mark obtained and most common progress among satisfactory,
    unsatisfactory and moderate.
    ---> input : stuIndex = Indices (row labels in the dataframe i.e. use .loc[stuIndex]) 
    of students who received a grade
    """
    stu_ID = dashbord["Student"].loc[stuIndex].values
    Marks = dashbord["Marks"].loc[stuIndex].values
    df = pd.DataFrame(stu_ID)
    df = df.rename(columns={0: "Student_ID"})
    df["Marks"] = Marks
    mostCommon = []#most commun progress (satisfactory,unsatisfactory,moderate) 
                  #for each student in the same order as stu_ID
    for i in range(len(stu_ID)):
        d = Counter(dashbord["progress"].iloc[np.where(dashbord["Student"]==stu_ID[i])].values)
        value, count = d.most_common()[0]
        mostCommon.append(value)

    df.insert(loc=2,column='mostCommonProg',value=mostCommon)
    return(df)

<font color = "green" > __To do__: 
- put percentage on the graph
- put 3 graphs on the same graph (/= curve?)
- look on the last n meetings 
- transform the classes in conintinous scale(transformer meetings in real value and rescale by the nb of meetings)
- 1 : satis, 0.5; moder ; 0: unsat puis sum and normalize. real value that summarizes the progress of a student; 
- boxplot : consistency in colors and order 

### computeProgressScore(...)

In [21]:
# Definition of a function score for the progress
#------------------------------------------------------------
def computeProgressScore(v):
    """
    This function takes as input a vector v = [v1, v2, v3] where 
    v1 = number of unsatisfactory progress,
    v2 = number of moderate progress,
    v3 = number of satisfactory progress
    for each supervisor or student and computes the satisfaction score.  
    
    The score is calculated as follows: 
    - if the progress is satisfactory we assign the weight: 1
    - if the progress is moderate we assign the weight: 2/3
    - if the progress is unsatisfactory we assign the weight: 1/3
    ProgressScore(v) = ( (1/3)*v1 + (2/3)*v2 + 1*v3 ) / (norm 1 of v)
    The higher the score, the higher the satisfaction.   
    """
    w = np.array([1/3, 2/3, 1]) #weights
    if np.all(v==0):
        s = 0
    else: 
        s = (w[0]*v[0]+w[1]*v[1]+w[2]*v[2])/np.sum(v)
    return(s)

### retrieve0Comments(...)

In [22]:
def retrieve0Comments(df):
    """
    This function retrieve the comments of students with 0 mark sorted by date (the last comment is the most recent)
    --- input: df(dataframe) = output of function computeProgress("Student")
    --- output: dfProgStu0(dataframe) = dataframe of statistics of students with 0 marks
        - unsatisfactory : proportion of unsatisfactory progress (among non NaN progress)
        - moderate : proportion of moderate progress (among non NaN progress)
        - satisfactory : proportion of satisfactory progress (among non NaN progress)
        - comments: comments of students with 0 mark sorted by date (the last comment is the most recent)
    """
    dfProgStu0 =  df.iloc[np.where(df["Marks"]==0)].copy()
    comments0 = [] #comments of students with 0 mark sorted by date (the last comment is the most recent)
    for s in dfProgStu0["Student_ID"]:
        comments0.append(completeDashbord[["comments","date"]].iloc[np.where(dashbord["Student"] == s)].sort_values(by="date")["comments"].values)
    dfProgStu0["comments"] = comments0
    dfProgStu0.index = dfProgStu0["Student_ID"]
    dfProgStu0.drop("Student_ID",axis=1,inplace = True)
    return(dfProgStu0)

### retrieveWhoFilledDur(...)

In [23]:
def retrieveWhoFilledDur(dfProgSup,durMoreThan,durLessThan,display = False):
    """
    This function retrieves the IDs of supervisors who filled more or less 
    than a given percentage of meetings' duration.
    ---> input: 
    - dfProgSup = computeProgress() only the columns 'Supervisor_ID' and '%FilledDuration' are used
    - durMoreThan (float): threshold to retrieve the IDs of supervisors who 
    filled "durMoreThan" or more of the meetings' duration
    - durLessThan (float): threshold to retrieve the IDs of supervisors who 
    filled "durLessThan" or less of the meetings' duration
    - display (boolean): if True, prints the IDs of supervisors who meet the criterias.
    ---> output:
    - ID_durMoreThan: IDs of supervisors who filled "durMoreThan" or more of the meetings' duration
    - ID_durLessThan: IDs of supervisors who filled "durLessThan" or less of the meetings' duration
    """
    ID_durMoreThan = dfProgSup["Supervisor_ID"].iloc[np.where(dfProgSup["%FilledDuration"] >= durMoreThan)].values
    ID_durLessThan = dfProgSup["Supervisor_ID"].iloc[np.where(dfProgSup["%FilledDuration"] <= durLessThan)].values
    
    if display == True:
        print("IDs of supervisors who filled" + Fore.GREEN + " {}% or more".format(durMoreThan) + Style.RESET_ALL + " of the meetings' duration:")
        print(ID_durMoreThan)
        print("")
        print("IDs of supervisors who filled" + Fore.RED + " {}% or less".format(durLessThan) + Style.RESET_ALL + " of the meetings' duration:")
        print(ID_durLessThan)    
    return ID_durMoreThan,ID_durLessThan

### computeDeclineTime(...)

In [24]:
def computeDeclineTime(dashbord):
    """
    This function computes for each student the time they have the first moderate or unsatisfactory meeting.
    - Only the progress of 'meetings' is considered (no 'approvals' or 'ethics')
    - The time to have the first unsatisfactory/moderate meeting is in days. 
      This is the difference between the start of the academic year and the date 
      of the first unsatisfactory/moderate progress
    - The academic year is considered to be the beginning of the lectures.
    ---> input: dashbord (dataframe): dashbord defined at the beginning of the notebook
    ---> output: dfDecline(dataframe): the columns are: 
    - Student_ID: Students IDs
    - declineDate: date of the first moderate or unsatisfactory meeting
    - daysToDecline: time (in days) between the first unsatisfactory/moderate progress 
                     and the beginning of the academic year   
    - nbMeetToDecline: number of meetings to have the first unsatisfactory/moderate progress  
    """
    stuList = dashbord["Student"].dropna().drop_duplicates().values

    #academic years = date of the start of lectures.
    #academic years: 2017-2018, 2018-2019, 2019-2020, 2020-2021, 2021-2022
    startLecture = {2017:"2017-09-18",2018:"2018-09-24",2019:"2019-09-23",2020:"2020-10-05",2021:"2021-09-20"}
    declineDate = []#list constaining the date of the first moderate or unsatisfactory 
                    #progress for each student in the same order as stuList
    daysToDecline = []#list containing the number of days between the start of the academic year and the first 
                      #unsatisfactory or satisfcatory progress
    nbToDecline = [] #list containing the number of meetings to fall into unsatisfactory/satisfactory progress

    for s in stuList:
        df_s = dashbord.iloc[np.where(dashbord["Student"]==s)].copy()#extract from "dashbord" the dataframe correspondig to student s
        df_s.sort_values(by="date",inplace = True)#sort values by ascending date
        df_s = df_s.iloc[np.where(df_s["type"]=="meeting")]#consider only 'meetings' and no 'approvals' or 'ethics'

        idxM = np.where(df_s["progress"]=="moderate")[0] #index in df_s of the first moderate progress
        idxU = np.where(df_s["progress"]=="unsatisfactory")[0]#index in df_s of the first unsatisfactory progress

        if (len(idxM)!=0 and len(idxU)!=0): #if there is a moderate and an unsatisfactory progress...
            idxFirst = min(idxM[0],idxU[0]) #... retrieve index in df_s of the fist moderate or unsatisfactory progress
        elif (len(idxM)==0 and len(idxU)!=0): #if there no moderate but an unsatisfactory progress...
            idxFirst = idxU[0] #... retrieve index in df_s of the first unsatisfactory progress
        elif (len(idxM)!=0 and len(idxU)==0):#if there a moderate but no unsatisfactory progress...
            idxFirst = idxM[0] #... retrieve index in df_s of the first moderate progress
        else: #if there is no moderate nor unsatisfactory progress    
            idxFirst = -1 #no indexes

        if idxFirst == -1: #if there is no unsatisactory or moderate progress
            declineDate.append(np.nan)
            daysToDecline.append(np.nan)
            nbToDecline.append(np.nan)
        else: #if there is an unsatisactory or moderate progress
            y = df_s["Year"].dropna().iloc[0] - 1 #year of the student s. -1 because we want the start date of the year.
            ay = dt.datetime.strptime(startLecture[y]+" 00:00:00", "%Y-%m-%d  %H:%M:%S") #date of start of the academic year for the year y
            d = df_s["date"].iloc[idxFirst] #date of the first moderate or unsatisfactory progress

            declineDate.append(d)
            daysToDecline.append((d-ay).days)#time (in days) between the first unsatisfactory/moderate progress and 
                                             #the beginning of the academic year for the student s    
            nbToDecline.append( idxFirst+1 )#number of meetings to have the first unsatisfactory/moderate progress

    #data formatting
    #--------------------------------------------------------
    dfDecline = pd.DataFrame(stuList.astype(int))
    dfDecline.rename(columns={0:"Student_ID"},inplace = True)
    dfDecline["declineDate"] = declineDate
    dfDecline["daysToDecline"] = daysToDecline
    dfDecline["nbMeetToDecline"] = nbToDecline
    return(dfDecline)

### computeDfPred(...)

In [85]:
def computeIrishGrade(m):
    """
    This function takes as input a mark out of 100 and returns the 
    corresponding grade according to Ireland's Academic grading:
    https://en.wikipedia.org/wiki/Academic_grading_in_the_Republic_of_Ireland
    """
    if m >= 90:
        grade = "H1"
    elif (m >= 80 and m <= 89):
        grade = "H2"
    elif (m >= 70 and m <= 79):
        grade = "H3"    
    elif (m >= 60 and m <= 69):
        grade = "H4"   
    elif (m >= 50 and m <= 59):
        grade = "H5"  
    elif (m >= 40 and m <= 49):
        grade = "H6"    
    else:
        grade = "N"
    return (grade)

In [86]:
# Retrieve the sentiment of the comments computed by the **best** BERT method
dfSentiment_3 = pd.read_pickle("data/dashData_from_2022-08-12_14_49_19_full_checkpoint.pkl") #3 labels: positive,neutral,negative
dfSentiment_2 = pd.read_pickle("data/dashData_from_2022-08-18_10_50_40_full_checkpoint.pkl") #2 labels: positive,negative

# Add a column "sentiment" to the "dashbord" dataframe 
#-- 3 labels
dashbordS_3 = dashbord.copy()
dashbordS_3["sentiment"] = np.zeros(len(dashbordS_3))*np.nan
dashbordS_3.sentiment.loc[dfSentiment_3.index] = dfSentiment_3.BERTlabel_1

#-- 2 labels
dashbordS_2 = dashbord.copy()
dashbordS_2["sentiment"] = np.zeros(len(dashbordS_2))*np.nan
dashbordS_2.sentiment.loc[dfSentiment_2.index] = dfSentiment_2.BERTlabel_1

C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\173074957.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dashbordS_3.sentiment.loc[dfSentiment_3.index] = dfSentiment_3.BERTlabel_1
C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\173074957.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dashbordS_2.sentiment.loc[dfSentiment_2.index] = dfSentiment_2.BERTlabel_1


In [37]:
beginAcademicYear = {2018:"2017-09-18",2019:"2018-09-24",2020:"2019-09-23",2021:"2020-10-05",2022:"2021-09-20"}

def computeDfPred(dashbord,beginAcademicYear,irishGrades=True):
    """
    This function computes the dataframe containing all information to predict the student's mark. 
    ---> input:
    - dashbord (dataframe): "dashbord" defined at the beginning of the Notebook where a column 
        containing the sentiment of each comment (predicted by BERT) has been added.
    - academicYear (dict): dictionary containing the the year of students' graduation and the 
        corresponding beginning of the academic year. For example, for students graduating in
        year "2018", the beginning of the academic year is "2017-09-18" so "2018" is associated
        with "2017-09-18" in the dictionary.
    - irishGrades (bool): if True, then the mark out of 100 is converted to Ireland's Academic grading. 
    ---> output: newDash (dataframe): where each row represent a Student ID and columns are: 
    - Supervisor (int): Supervisor IDs. One student ID can have several supervisors IDs. 
        For each student, we keep the first supervisor IDs if several supervisor IDs are found. 
    - nbMeetings (int): number of meetings for each Student
    - worstProgress (category): worst progress for each Student among "ethics", "meetings","approval". 
        * If a student has at least 1 unsatisfactory progress, they fall into the unsatisfactory box
        * If a student has 0 unsatisfactory but at least 1 moderate progress, they fall into the moderate box
        * If a student has 0 unsatisfcatory and 0 moderate progress, they fall into the satisfactory box
    - meanTimeBetweenMeet (float): mean time between meetings for each Student in days. Only "meeting" 
        are considered (no "approval" or "ethics"). 
        Mean time = (end date meetings - start date meetings)/number of meetings 
    - meanTimeMeet (int): mean time of a meeting in minutes. If for a student, there is no meetings or 
        no duration filled, the value of will be NaN. 
    - timeFirstDecay (int): time in days between the beginning of the academic year 
        and the first 'unsatisfactory'/'moderate' "meeting" (no "ethics" or "approval").
        If there is no meeting for the student, we put this value to NaN. 
    - mainSentiment (int): most common sentiment of a the comments considering only comments from "meeting"
        (no "ethics" or "approval"). If there is no comments or no "meeting" for the student, the value is NaN.
    - is0Sentiment (bool): contains True if the is a comment's sentiment at 0 (negative) and False otherwise. 
     Considers only comments from "meeting"(no "ethics" or "approval"). If there is no comments or 
     no "meeting" for the student, the value is NaN.
    """

    newDash = dashbord[["Student","Marks","module","Year","Supervisor"]].drop_duplicates() #conserve only relevant information for student statistics
    newDash.dropna(axis=0,inplace=True) #remove rows with NaN values which are:
                                        #"Student" and "Marks" values both missing at the same rows
                                        #"Supervisor" values missing in general at different rows from "Student" and "Marks"    
    idxDup = np.where(newDash.Student.duplicated(keep='first'))  #indices in "newDash" of duplicated student ID (for students who have 2 supervisors)
                                                                 #keep='first': Mark duplicates as True except for the first occurrence.
    newDash.drop(newDash.iloc[idxDup].index,axis=0,inplace=True) #keep only the first supervisor if several supervisors IDs are found for 1 student ID


    nbMeetings = [] #number of meetings for each Student
    worstProgress = [] #worst progress for each Student 
    meanTimeBetweenMeet = [] #mean time between meetings for each Student
    meanTimeMeet = [] #mean time of a meeting for each Student
    timeFirstDecay = [] #time between the beginning of the year and the first decay in progres 
    mainSentiment = [] #main sentiment of a the comments
    is0Sentiment = [] #contains True if the is a comment's sentiment at 0 (negative)
    
    i=0
    for s in newDash["Student"]:
        # if irishGrades = True, replace the mark by the Irish grade system
        if irishGrades:
            newDash.Marks.iloc[i] = computeIrishGrade(newDash.Marks.iloc[i])
            #### This line generates a WARNING ####

        idx = np.where(dashbord.Student == s) #indices in "dashbord" where Student = s
        #=================================
        # Number of meetings for student s. Is 0 if no meeting found. 
        #=================================
        nbMeet_s = Counter(dashbord.type.iloc[idx])['meeting']
        nbMeetings.append(nbMeet_s)
        #=================================
        # Worst progress for student s
        #=================================
        #-- if student s has at least 1 'unsatisfactory' progress:
        if 'unsatisfactory' in Counter(dashbord.progress.iloc[idx]):
            worstProgress.append('unsatisfactory')
        #-- else if student s has at least 1 'moderate' progress:
        elif 'moderate' in Counter(dashbord.progress.iloc[idx]):
            worstProgress.append('moderate')
        #-- else if student s has only 'satisfactory' progress:
        elif 'satisfactory' in Counter(dashbord.progress.iloc[idx]):
            worstProgress.append('satisfactory')
        #-- else put worst progress to NaN:
        else:
            worstProgress.append(np.nan)

        #=================================
        # Mean time between meetings for student s 
        #=================================
        idxMeetings = np.where(dashbord.type.iloc[idx]=="meeting") #indices of meetings in the "dashbord" reduced at Student s 
        dash_s_meet = dashbord.iloc[idx].iloc[idxMeetings] #"dashbord" reduced at student s with only "meeting" type
        dash_s_meet.sort_values(by="date",inplace=True)# sort by date 
        # if there is at least one meeting for student s
        if len(dash_s_meet) != 0:
            startDate = dash_s_meet.date.iloc[0] #date of the first meeting
            endDate = dash_s_meet.date.iloc[-1]  #date of the last meeting
            meanTimeBetweenMeet.append(round((endDate-startDate).days/nbMeet_s)) #append mean time between meetings (rounded value)
        # if there is no meetings for student s, mean time between meetings is 0
        else:
            meanTimeBetweenMeet.append(0) 

        #=================================
        # Mean time of a meeting for student s
        #=================================
        meanTimeMeet.append(np.mean(dash_s_meet.duration))

        #=================================
        # Time first decay for student s
        #=================================
        # if there is at least one meeting for student s
        if len(dash_s_meet) != 0:
            beginningYear = beginAcademicYear[dash_s_meet.Year.iloc[0]] #beginning of the academic year for student s
            beginningYear = dt.datetime.strptime(beginningYear,"%Y-%m-%d") #convert string into timeStamp

            IfirstModer = dash_s_meet[dash_s_meet.progress == "moderate"].first_valid_index() #index of the first 'moderate' progress or None
            IfirstUnsat = dash_s_meet[dash_s_meet.progress == "unsatisfactory"].first_valid_index() #index of the first 'unsatisfactory' progress or None

            # if there is a 'moderate' and a 'unsatisfactory' progress
            if IfirstUnsat and IfirstModer:
                dateFirstDecay = min(dash_s_meet.date.loc[IfirstModer],dash_s_meet.date.loc[IfirstUnsat])
            # if there is a 'moderate' but no'unsatisfactory' progress
            elif IfirstModer: 
                dateFirstDecay = dash_s_meet.date.loc[IfirstModer]
            # if there is a 'unsatisfactory' but no 'moderate' progress
            elif IfirstUnsat:
                dateFirstDecay = dash_s_meet.date.loc[IfirstUnsat]
            else:
                dateFirstDecay = beginningYear
            timeFirstDecay.append((dateFirstDecay - beginningYear).days)
        else:
            timeFirstDecay.append(np.nan)

        #=================================
        # Most common sentiment of the meeting comments of student s 
        # and indicates if the meetings contains a 0 (negative) sentiment
        #=================================
        idxIsSent = np.where(dashbord.sentiment.iloc[idx].iloc[idxMeetings].isna()==False)
        #indices of non-NaN sentiment among "dashbord" reduced to student s and only meetings 
        dash_sent = dashbord.iloc[idx].iloc[idxMeetings].iloc[idxIsSent] #recuced "dashbord" constructed with idxIsSent
        if len(dash_sent) != 0: #if there is at least one sentiment (ie one meeting comment)
            list_sent = list(dash_sent.sentiment.values) #all sentiments in "dash_sent" 
            mainSentiment.append(max(set(list_sent), key = list_sent.count))#append most common sentiment
            if Counter(dash_sent.sentiment)[0] != 0: #if there is a 0 (negative) comment
                is0Sentiment.append(True) #append True
            else: #else append False
                is0Sentiment.append(False)
        else: #else append NaN
            mainSentiment.append(np.nan)
            is0Sentiment.append(np.nan)
        
        i+=1

    #=================================
    # Add computed information to newDash
    #=================================
    newDash["nbMeetings"] = nbMeetings
    newDash["worstProgress"] = worstProgress 
    newDash["meanTimeBetweenMeet"] = meanTimeBetweenMeet
    newDash["meanTimeMeet"] = meanTimeMeet
    newDash["timeFirstDecay"] = timeFirstDecay
    newDash["mainSentiment"] = mainSentiment
    newDash["is0Sentiment"] = is0Sentiment

    #=================================
    # Assign correct type to variables
    #=================================
    newDash.Marks = pd.Categorical(newDash.Marks,ordered=False)
    newDash.module = pd.Categorical(newDash.module,ordered=False)
    #newDash.Year = pd.Categorical(newDash.Year,ordered=False)
    newDash.worstProgress = pd.Categorical(newDash.worstProgress,ordered=False)
    newDash.Supervisor = newDash.Supervisor.astype(int)
    newDash.Student = newDash.Student.astype(int)
    newDash.mainSentiment = pd.Categorical(newDash.mainSentiment,ordered=False)
    newDash.is0Sentiment = pd.Categorical(newDash.is0Sentiment,ordered=False)
    
    return (newDash)

One student ID can have 2 supervisors: how can I decide which supevisor should I consider?
For the moment, I will just take 1 supervisor ID among the 2

```
dashbord.iloc[np.where(dashbord.Student == 318)]
```

____________
____________
<font color = "purple" > **This part need to be removed** </font>

In [ ]:
"""# Split between train set and validation set
from sklearn.model_selection import train_test_split  

X = dfPred.drop(labels = "Marks",axis=1).copy() #explanatory variables
y = dfPred["Marks"] #response variable

# IMPUTAION HERE


# RandomForestClassifier does not accept strings. 
# Thus, we split out columns of categorical data into sets of boolean columns, 
# one new column for each unique value in each input column
X = pd.get_dummies(X)

X_train, X_val, y_train, y_val =  train_test_split(X, y, test_size=0.1, random_state=2022)"""

A common step is a **data standardisation or normalisation** step. The variables are divided by their standard deviation. This is not useful in the case of an elementary linear model because the solution is the same, but it is essential for many other non-linear methods (SVMs, neural networks, models with penalties). This step is therefore systematically carried out in practice to avoid problems. Warning:  the same parameters (means, standard deviations) estimated on the training sample are used to normalise the test sample. (*source: my Machine Learning course*) \
However, according to : https://stackoverflow.com/questions/8961586/do-i-need-to-normalize-or-scale-data-for-randomforest-r-package <font color="red"> **Random Forest does not require feature scaling.**</font>

Apparently, RandomForestClassifier cannot handle missing (NaN) values. The reason is beacause this RF algorithm is not build on CART trees (which we know can handle missing values) but on other decision trees for computation reasons: https://stats.stackexchange.com/questions/98953/why-doesnt-random-forest-handle-missing-values-in-predictors \
We need to imput data.

**Method 1:** Simple method consisting to replace missing values with the mean

Imputer: fit Transform method.
https://towardsdatascience.com/fit-vs-transform-in-scikit-libraries-for-machine-learning-3c70e6300ded

example of using here: https://stackoverflow.com/questions/30317119/classifiers-in-scikit-learn-that-handle-nan-null

library for imputation: https://scikit-learn.org/stable/modules/impute.html

In [ ]:
"""from sklearn.impute import SimpleImputer
# Create our imputer to replace missing values with the mean:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute our data
X_train_imp = imp.transform(X_train)
X_val_imp = imp.transform(X_val)"""

In [ ]:
"""#if we use the Irish Grade system for the marks, we are in the case of classification
from sklearn.ensemble import RandomForestClassifier 

# definition of parameters
forest = RandomForestClassifier(n_estimators=500, 
                                criterion='gini', max_depth=None,
                                min_samples_split=2, min_samples_leaf=1, 
                                max_features='auto', max_leaf_nodes=None,
                                bootstrap=True, oob_score=True)
#help(RandomForestClassifier)
rfFit = forest.fit(X_train,y_train)"""

<font color = "purple" > **End of the part to remove** </font>
____________
____________

## Calling compute functions

In [39]:
#retrieveInfo()
#------------------------------------------------
stuIndex,nbStuG,supIDG = retrieveInfo()

In [40]:
#computeProgress(...)
#------------------------------------------------
dfProgSup = computeProgress()
dfProgStu = computeProgress("Student")
dfProgSup_s = computeProgress(stacked=True)#if we want to plot a stacked Histogram for Supervisors
dfProgStu_s = computeProgress("Student",stacked=True)#if we want to plot a stacked Histogram for Students

In [41]:
#computeWorstProgress()
#------------------------------------------------
dfUnsat,dfModer,dfSatis = computeWorstProgress()

C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\782746031.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfProgStu_n["unsatisfactory_box"].iloc[Ifall_u] = 1 #put value 1 for indexes Ifall_u
C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\782746031.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfProgStu_n["moderate_box"].iloc[Ifall_m] = 1 #put value 1 for indexes Ifall_m
C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\782746031.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [42]:
#computeModule(...)
#------------------------------------------------
dfProgSup_m = computeModule(dfProgSup,dashbord)
dfProgSup_s_m = computeModule(dfProgSup_s,dashbord)

In [43]:
#computeMostCommonProgress(...)
#------------------------------------------------
dfMostCommon=computeMostCommonProgress(stuIndex)

In [44]:
#retrieve0Comments
#------------------------------------------------
dfProgStu0 = retrieve0Comments(dfProgStu)

In [45]:
#computeDeclineTime(...)
#------------------------------------------------
dfDecline = computeDeclineTime(dashbord)

In [47]:
#computeDfPred()
#------------------------------------------------
dfPredC_3 = computeDfPred(dashbordS_3,beginAcademicYear) #3 labels: positive,neutral,negative
dfPredC_2 = computeDfPred(dashbordS_2,beginAcademicYear) #2 labels: positive,negative

C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\3018113000.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDash.Marks.iloc[i] = computeIrishGrade(newDash.Marks.iloc[i])
C:\Users\coxi\AppData\Local\Temp\ipykernel_14112\3018113000.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDash.Marks.iloc[i] = computeIrishGrade(newDash.Marks.iloc[i])


<font color = "purple" > **Intructions to be removed:** </font>

Transform categ var into dummy variables

worst Progress in the last 5 meetings: 
if less than 5 meetings: take wathever is available
or progress of last meeting. 

SHAP python: provide explanation of NN decisions (importance of features in the final decision)

run several exp: RF,NN,logistic Regression and find best model
on the best model: try tests to see if I can remove variables: significant tests

30 comments: for tuning meta parameters (datasets used, epochs, lr): find best/interesting models (do not wrtie which one in the models) just write the overall procedure
50 comments: for testing the best models found on the 30 comments --> in the report 
make recommendation for the best model for prediction.

report:
sentiment discord between progresss and sentiment
comments are /= from tweets

then talk prediction: try to predict 
further reaserch: prediction on small window 

In [84]:
#pickle the results for latter use
#------------------------------------------------
dfProgSup.to_pickle("result/dfProgSup.pkl")
dfProgStu.to_pickle("result/dfProgStu.pkl")
dfProgSup_s.to_pickle("result/dfProgSup_s.pkl")
dfProgStu_s.to_pickle("result/dfProgStu_s.pkl")
#dfPredC_3.to_pickle("result/dfPredC_3.pkl")
#dfPredC_2.to_pickle("result/dfPredC_2.pkl")
#export to xlsx file for later use
#------------------------------------------------
dfPredC_3.to_excel("result/dfPredC_3.xlsx")
dfPredC_2.to_excel("result/dfPredC_2.xlsx")

## Plots: data visualisation

### plotModule(...) & plotMarkPerYear(...)

In [ ]:
#plot de number of students in each module throughout the years
#---------------------------------------------------------------
def plotModule(dashbord,onlyUnsat=False):
    """
    ---> input: 
    - onlyUnsat (boolan): if True, only plots the number of students per module for each year 
    who had at least 1 unsatisfactory progress. If False, plots the number of students 
    per module for each year for all progresses (satisfactory/unsatisfactory/moderate)
    - dashbord (dataframe): if onlyUnsat=False it is the main 'dashbord' defined at the 
    beginning of the notebook . If onlyUnsat=True, it is 'dfUnsat' given by the function
    'computeWorstProgress()'   
    """
    
    if onlyUnsat == False:
        name = "Student"
        title = "Number of students per module for each year"
    else: 
        name = "Student_ID" #adapt column name to dataframe 'dfUnsat' instead of dataframe 'dashbord'
        title = "Number of students who have " + r"$\bf{at~least~1~unsatisfactory~progress}$ per module for each year"
        
    modules = np.sort(dashbord["module"].drop_duplicates().values)
    years = np.sort(dashbord["Year"].drop_duplicates().values)
    nbStuMod = np.zeros((len(years),len(modules))) #number of students per module per yeas. 
    #Rows: years in the same order as variable years
    #Columns: modules in the same order as variable module

    for i in range(len(years)):
        for j in range(len(modules)):
            idxY = np.where(dashbord["Year"] == years[i])#indexes where the year = years[i] (use iloc[])
            idxM = np.where(dashbord["module"].iloc[idxY] == modules[j])#in year years[i], indices where module = modules[j (use .iloc[])
            idxSt = dashbord[name].iloc[idxY].iloc[idxM].drop_duplicates().values
            idxSt = delete(idxSt,np.where(np.isnan(idxSt)))#delete nan value        
            nbStuMod[i,j] = len(idxSt)

    #--- data formatting
    dfModule = pd.DataFrame(nbStuMod)
    dfModule.columns = modules
    dfModule.index = years

    #plot of the graph
    #---------------------------------------------------------------
    f, ax = plt.subplots(figsize=(15, 6))

    #---number of students per year per module
    for i in range(len(modules)):
        sns.lineplot(ax = ax, x = years, y = modules[i], data = dfModule,
                 label = '{}'.format(modules[i]), color = sns.color_palette("tab10")[i])
        sns.scatterplot(ax = ax, x = years, y = modules[i], data = dfModule,color = sns.color_palette("tab10")[i])
        #add mean mark value above each point 
        for j,v in enumerate(dfModule[modules[i]]):
            ax.text(dfModule[modules[i]].index[j], v+1, "%d" %v, ha="center",color = sns.color_palette("tab10")[i], size = 12)

    ax.set(xlabel="years", ylabel="number of students")
    ax.set_title(title,pad=30,fontsize = 14)
    ax.set_xticks(years)
    
    #--- background color
    ax.patch.set_facecolor('grey')
    ax.patch.set_alpha(0.05)
    plt.grid(linestyle = 'dotted')

    sns.despine(left = True, bottom = True) #remove graph frame
    plt.show()

In [ ]:
def plotMarkPerYear(df,median=False):
    """
    This function plots the mean mark per module per year. If median = True,
    the function plots the median instead
    ---> input: 
    - df (dataframe) = output of function computeProgress("Student")
    - median (bool): if false, compute the mean mark per module per year. 
    if true, compute the median instead.
    ---> output: plot
    """
    modules = np.sort(dashbord["module"].drop_duplicates().values)
    years = np.sort(dashbord["Year"].drop_duplicates().values)
    meanMark = np.zeros((len(years),len(modules))) #mean mark per module per year. 
    #Rows: years in the same order as variable years
    #Columns: modules in the same order as variable module

    
    for i in range(len(years)):
        for j in range(len(modules)):
            idxY = np.where(df["Year"]==years[i]) #indexes where year = years[i] 
            idxY_M = np.where(df["module"].iloc[idxY]==modules[j]) #among year years[i], indexes where module = modules[j] 
            if median == False:
                meanMark[i,j] = df["Marks"].iloc[idxY].iloc[idxY_M].mean()
            else:
                meanMark[i,j] = df["Marks"].iloc[idxY].iloc[idxY_M].median()
                
    #--- data formatting
    dfMeanMark = pd.DataFrame(meanMark)
    dfMeanMark.columns = modules
    dfMeanMark.index = years        

    #plot of the graph
    #---------------------------------------------------------------
    f, ax = plt.subplots(figsize=(15, 5))

    #---number of students per year per module
    for i in range(len(modules)):
        sns.lineplot(ax = ax, x = years, y = modules[i], data = dfMeanMark,
                 label = '{}'.format(modules[i]), color = sns.color_palette("tab10")[i])
        sns.scatterplot(ax = ax, x = years, y = modules[i], data = dfMeanMark,color = sns.color_palette("tab10")[i])
        #add mean mark value above each point 
        for j,v in enumerate(dfMeanMark[modules[i]]):
            if np.isnan(v) == False :
                ax.text(dfMeanMark[modules[i]].index[j]-0.01, v+0.5, "%d" %v, ha="center",color = sns.color_palette("tab10")[i], size = 12)
    
    if median == False:
        name = "Mean mark"
    else:
        name = "Median mark"
        
    ax.set(xlabel="years", ylabel=name)
    ax.set_title(name + ' per module for each year',pad=30,fontsize = 14)
    ax.set_xticks(years)

    #--- background color
    ax.patch.set_facecolor('grey')
    ax.patch.set_alpha(0.05)
    plt.grid(linestyle = 'dotted')
    sns.despine(left = True, bottom = True) #remove graph frame
    plt.show()

In [ ]:
plotModule(dashbord)
plotModule(dfUnsat,onlyUnsat=True)
plotMarkPerYear(dfProgStu,median=True)
plotMarkPerYear(dfProgStu,median=False)

<font color = red> There is no student in mcm module for the year 2022 because all students in year 2022 and module mcm have no student ID.

### boxplot Progress~Duration

In [ ]:
#Boxplot progress~duration
#-----------------------------------
#--- method 1
Idur = np.where(dashbord["duration"].isna()!= True) #indexes where there is a duration
Dur = dashbord["duration"].iloc[Idur].values
Prog = dashbord["progress"].iloc[Idur].values
df = pd.DataFrame(Dur)
df = df.rename(columns={0:"duration"})
df["progress"] = Prog
f, (ax1,ax2) = plt.subplots(1,2,figsize=(15, 5))
flierprops = dict(markerfacecolor='0.75', markersize=4,linestyle='none') #size & color of outliers
sns.boxplot(ax=ax1,x="progress", y="duration", data=df, flierprops=flierprops,linewidth=1, 
            showfliers=False,palette = ['slateblue','green','red'])
ax1.set_title("boxplot progress ~ duration with method 1")
#--- method 2 : same result as method 1...
sns.boxplot(ax=ax2, x="progress", y="duration", data=dashbord, flierprops=flierprops,linewidth=1,
            showfliers=False,palette = ['slateblue','green','red'])
ax2.set_title("boxplot progress ~ duration with method 2")
plt.show()

### scatterplot Mark~Duration

In [ ]:
#Scatter plot of duration as a function of Marks
#----------------------------------------------------------
f,ax = plt.subplots(figsize=(15, 5))
sns.scatterplot(ax=ax,x="Marks",y="duration",data=dashbord,s=20)
ax.set_ylim(0,150)
ax.set_xticks(np.linspace(0,100,25).astype(int))
sns.despine(left = True, bottom = True) #remove graph frame
ax.set_title("Scatter plot of duration as a function of Marks")
ax.set_ylabel("duration (min)")
ax.patch.set_facecolor('grey')#background color
ax.patch.set_alpha(0.05)#background color
plt.grid(linestyle='dotted')

<font color = 'red'> Some supervisors put a meeting with 0 value for example, when they added comments after the actual meeting.

### Duration histogram

In [ ]:
#plot of the duration histogram
#----------------------------------------------
#--- complete histogram
f, (ax1,ax2) = plt.subplots(1,2,figsize=(15, 5))
sns.histplot(ax= ax1, data = dashbord["duration"],stat='proportion',
             binwidth = 1,color = 'steelblue', edgecolor = 'w')
ax1.set_title ("Histogram of the duration")
ax1.set_ylabel("duration (proportion)")
ax1.set_xlim(0,200)

#--- zoom on the area of interest
sns.histplot(ax= ax2, data = dashbord["duration"],stat='proportion',
             binwidth = 1,color = 'steelblue', edgecolor = 'w')
xlim1 = 0; xlim2 = 100
ylim1 = 0; ylim2 = 0.15
ax2.set_title ("Histogram of the duration\nzoom around x=[{},{}] and y=[{},{}]".format(xlim1,xlim2,ylim1,ylim2))
ax2.set_ylabel("duration (proportion)")
ax2.set_xlim(xlim1,xlim2)
ax2.set_ylim(ylim1,ylim2)

sns.despine(left = True, bottom = True) #remove graph frame
plt.show()

### Mark histogram,boxplot,QQ-plot

In [ ]:
#plot of the Mark histogram, boxplot and QQ-plot
#----------------------------------------------
Marks = dfProgStu["Marks"]

f, (ax1,ax2) = plt.subplots(1,2,figsize=(15, 5),gridspec_kw={'width_ratios': [3, 1]})
sns.histplot(ax=ax1,data = Marks,stat='proportion',
             binwidth = 1.5,color = 'steelblue', edgecolor = 'w')
ax1.set_xticks(np.linspace(0,100,25).astype(int))
ax1.set_ylabel ("Proportion of Marks")
ax1.set_title ("Histogram of the Marks")

sns.boxplot(ax=ax2,y=Marks,flierprops=flierprops,linewidth=1)
ax2.set_title ("Boxplot of the Marks")

sns.despine(left = True, bottom = True) #remove graph frame
#plt.savefig("Marks_dispersion.jpeg")
plt.show()

fig = plt.subplots(figsize=(8, 5))
stats.probplot(Marks, dist="norm", plot=pylab)
plt.title("QQ-plot of variable Marks")
#plt.savefig("QQ-plot_Marks.jpeg")
pylab.show()

In [ ]:
Marks.describe()

In [ ]:
#remove from variable Marks the Marks = 0
#--------------------------------------------------
idx0Marks = np.where(Marks==0) #indexes where the Marks = 0
n0Marks = Marks.drop(index=Marks.iloc[idx0Marks[0]].index.tolist())#Marks where the 0-marks have been removed 

print("percentage of 0-marks: {}%".format(round( len(Marks.iloc[idx0Marks])/ len(Marks)*100,3)))

In [ ]:
#Plot of the new variable Marks without the 0 marks denoted non0Marks
#--------------------------------------------------
f, (ax1,ax2) = plt.subplots(1,2,figsize=(10, 5),gridspec_kw={'width_ratios': [1, 3]})
sns.boxplot(ax=ax1,y=n0Marks,  flierprops=flierprops,linewidth=1)
stats.probplot(n0Marks, dist="norm", plot=pylab)
ax1.set_title("Boxplot of variable Marks \n without 0 and NaN marks")
ax2.set_title("QQ-plot of variable Marks without 0 and NaN marks")
#plt.savefig("QQ-plot_Marks_without_0_values.jpeg")
pylab.show()

### Mark histogram, boxplot per module

In [ ]:
#plot of the Mark histogram and boxplot for each module
#----------------------------------------------
#--- Computation
modulesList = dfProgStu["module"].drop_duplicates().values #list of modules names
marksPerMod = [] #list containing the list of marks for each module (in the same order as modulesList)

for m in modulesList:
    marksPerMod.append(dfProgStu["Marks"].iloc[np.where(dfProgStu["module"]==m)].values)

#--- graphs
f, axs = plt.subplots(5,2,figsize=(15, 20),gridspec_kw={'width_ratios': [4,1]})
axs = axs.ravel()
l1 = np.arange(0,10,2)
l2 = np.arange(1,10,2)
colors = sns.color_palette("tab10")

for i in range(len(l1)):
    sns.histplot(ax=axs[l1[i]],data = marksPerMod[i],stat='proportion',
             binwidth = 1.5,color = colors[l1[i]], edgecolor = 'w')
    axs[l1[i]].set_xticks(np.linspace(0,100,25).astype(int))
    axs[l1[i]].set_ylabel ("Proportion of Marks")
    axs[l1[i]].set_xlabel ("Marks")
    axs[l1[i]].set_title ("Histogram of the Marks for module "+modulesList[i])
    
    sns.boxplot(ax=axs[l2[i]],y=marksPerMod[i],flierprops=flierprops,linewidth=1,color = colors[l1[i]])
    axs[l2[i]].set_title ("Boxplot of the Marks for module "+modulesList[i])

    sns.despine(left = True, bottom = True) #remove graph frame
f.tight_layout()
#plt.savefig("Marks_dispersion_per_module.jpeg")
plt.show()

### boxplotMarksPerModule(...)

In [ ]:
def boxplotMarksPerModule(dfProgStu,dashbord):
    #----- 1. creates a dataframe containing the marks for each module
    modList = dashbord["module"].drop_duplicates().values
    MarkMod = [] #list containing the list of grades for each module in the same order as modList

    for m in modList:
        MarkMod.append(dfProgStu["Marks"].iloc[np.where(dfProgStu["module"]==m)].values) #add marks for module m
    dfMarkMod = pd.DataFrame(MarkMod).T #transform MarkMod into a dataframe 
    for i in range(len(modList)):
        dfMarkMod = dfMarkMod.rename(columns = {i:modList[i]}) #rename columns according to module name

    #----- 2. boxplot of marks for each module
    f, ax = plt.subplots(1,1, figsize=(10,5))
    sns.set_color_codes('pastel')
    flierprops = dict(markerfacecolor='0.75', markersize=4,linestyle='none') #size & color of outliers
    #--boxplot
    sns.boxplot(ax=ax, data = dfMarkMod,flierprops=flierprops,linewidth=1,palette = sns.color_palette("tab10"))
    #--axes
    ax.set(ylabel="mark dispersion", xlabel="module")
    ax.set_title("boxplot of students' marks for each module")
    sns.despine(left = True, bottom = True) #remove graph frame

In [ ]:
boxplotMarksPerModule(dfProgStu,dashbord)

### plotMostCommonProgress(...)

In [ ]:
def plotMostCommonProgress(df):
    """
    ---> input: dataframe returned by the function mostCommonProgress(stuIndex)
    """
    Isatif = np.where(df["mostCommonProg"] == 'satisfactory')#indexes where the progress is satisfactory
    marksS = df["Marks"].iloc[Isatif] #mark associated
    Imoder = np.where(df["mostCommonProg"] == 'moderate')#indexes where the progress is satisfactory
    marksM = df["Marks"].iloc[Imoder] #mark associated
    Iunsat = np.where(df["mostCommonProg"] == 'unsatisfactory')#indexes where the progress is satisfactory
    marksU = df["Marks"].iloc[Iunsat] #mark associated
    
    f, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15, 5))
    sns.histplot(ax=ax1,data = marksU,stat='proportion',
             binwidth = 1,color = 'r', edgecolor = 'w',label='unsatisfactory')
    ax1.set_title("For each mark, plot of the frequency of \n students whose majority progress is unsatisfactory")
    ax1.set_xlabel("Marks obtained")
    ax1.set_ylabel("frequency of students with unsatisfactory as majority class")
    sns.histplot(ax=ax2,data = marksM,stat='proportion',
             binwidth = 1,color = 'steelblue', edgecolor = 'w')
    ax2.set_title("For each mark, plot of the frequency of \n students whose majority progress is moderate")
    ax2.set_xlabel("Marks obtained")
    ax2.set_ylabel("frequency of students with moderate as majority class")
    sns.histplot(ax=ax3,data = marksS,stat='proportion',
             binwidth = 1,color = 'g', edgecolor = 'w')
    ax3.set_title("For each mark, plot of the frequency of \n students whose majority progress is satisfactory")
    ax3.set_xlabel("Marks obtained")
    ax3.set_ylabel("frequency of students with satisfactory as majority class")
    sns.despine(left = True, bottom = True) #remove graph frame
    plt.tight_layout()
    plt.show()
    
    #Boxplot progress~Marks
    #--------------------------------------------------------
    f, (ax1,ax2) = plt.subplots(1,2,figsize=(10, 5),gridspec_kw={'width_ratios': [2,1]})
    flierprops = dict(markerfacecolor='0.75', markersize=4,linestyle='none') #size & color of outliers
    sns.boxplot(ax=ax1,x="progress", y="Marks", data=dashbord, flierprops=flierprops,linewidth=1,
               order = ["unsatisfactory","moderate","satisfactory"],palette=["red","slateblue","green"],
                boxprops=dict(alpha=.5))
    
    ax1.set_title("Boxplot Progress ~ Marks")
    text = "Note that a student can fall into several boxes "
    ax2.text(0.1, 0.5, text, horizontalalignment='left', transform=ax2.transAxes,fontsize=12)
    ax2.axes.get_xaxis().set_visible(False)
    ax2.axes.get_yaxis().set_visible(False) 
    sns.despine(left = True, bottom = True) #remove graph frame
    subplots_adjust(wspace=0.0)
    plt.show()

#Documentation (subplots loop)
#https://stackoverflow.com/questions/17210646/python-subplot-within-a-loop-first-panel-appears-in-wrong-position

In [ ]:
plotMostCommonProgress(dfMostCommon)

### BoxplotWorstProgress(...)

In [ ]:
def boxplotWorstProgress(dfU,dfM,dfS):
    """
    This function plots a Boxplot of Marks as a function of the worst progress for the students.
    ---> input: dfU,dfM,dfS (dataframes) which are the output of the function ComputeWorstProgress()
    ---> output: Boxplot of Marks ~ Worst progress
    """
    f, ax = plt.subplots(figsize=(8, 5))

    #----- plot of the 3 boxplots
    c = 'red'
    ax = dfU.boxplot(column = "Marks",positions = [0],
                     color=dict(boxes=c, whiskers=c, medians=c, caps=c),
                     boxprops=dict(linestyle='-',color=c,linewidth=3,alpha=0.5),
                     medianprops=dict(linestyle='-',color=c, linewidth=3),
                     showfliers=True,grid=False)
    c = 'blue'
    ax = dfM.boxplot(column = "Marks",positions = [1],
                     color=dict(boxes=c, whiskers=c, medians=c, caps=c),
                     boxprops=dict(linestyle='-',color=c,linewidth=3,alpha=0.5),
                     medianprops=dict(linestyle='-',color=c, linewidth=3),
                     showfliers=True,grid=False)
    c = 'green'
    ax = dfS.boxplot(column = "Marks",positions = [2],
                     color=dict(boxes=c, whiskers=c, medians=c, caps=c),
                     boxprops=dict(linestyle='-',color=c,linewidth=3,alpha=0.5),
                     medianprops=dict(linestyle='-',color=c, linewidth=3),
                     showfliers=True,grid=False)

    #----- Definition of axes
    plt.xticks([1], ['']) #remove original xlabels
    #--label 1
    text1 = "Unsatisfactory box \n - {} students -".format(len(dfU))
    ax.text(0.07, -0.13, text1, horizontalalignment='left', transform=ax.transAxes,fontsize=12,color = 'red')
    #--label 2
    text2 = "Moderate box \n- {} students -".format(len(dfM))
    ax.text(0.43, -0.13, text2, horizontalalignment='left', transform=ax.transAxes,fontsize=12,color = 'blue')
    #--label 3
    text3 = "Satisfactory box\n- {} students -".format(len(dfS))
    ax.text(0.77, -0.13, text3, horizontalalignment='left', transform=ax.transAxes,fontsize=12,color = 'green')

    #---- Remove all axis 
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    #---- Put ylabel
    ax.set_ylabel("Marks",fontsize=12)

    #---- Background, tile and grid
    plt.grid(linestyle='dotted') #grid
    ax.patch.set_facecolor('grey')#background color
    ax.patch.set_alpha(0.05)#background color
    plt.suptitle("Boxplot of Marks ~ Worst progress",fontsize=14) #title

    plt.show()

    text4 = "Construction of the boxplots:\n\n- If a student has at least 1 unsatisfactory progress, they fall into the unsatisfactory box\n\n- If a student has 0 unsatisfactory but at least 1 moderate progress, they fall into the moderate box\n\n- If a student has only satisfactory progress they fall into the satisfactory box"
    print("\n\n"+text4)

# documentation:
# https://stackoverflow.com/questions/35160956/pandas-boxplot-set-color-and-properties-for-box-median-mean         

In [ ]:
boxplotWorstProgress(dfUnsat,dfModer,dfSatis)

### plotProgressScoreStudent()

In [ ]:
def plotProgressScoreStudent():
    """
    This function plots the Progress Score as a function of the Marks for each student
    """
    #computation
    #-------------------------------------------
    dfProgStu_ = computeProgress(name = "Student",scale = False)
    score = []
    for i in range(len(dfProgStu_)):
        v = dfProgStu_[["unsatisfactory","moderate","satisfactory"]].iloc[i].values
        score.append(computeProgressScore(v))     
    dfProgStu_["progressScore"] = score     
    
    #graph
    #-------------------------------------------
    f,ax = plt.subplots(figsize=(15, 5))
    sns.scatterplot(x = "Marks",y="progressScore",data=dfProgStu_,s=20)
    sns.despine(left = True, bottom = True) #remove graph frame
    ax.set_title("Scatter plot of progress score as a function of Marks")
    ax.set_ylabel("progress score")
    ax.set_xticks(np.linspace(0,100,25).astype(int))
    ax.patch.set_facecolor('grey')#background color
    ax.patch.set_alpha(0.05)#background color
    plt.grid(linestyle='dotted')

In [ ]:
plotProgressScoreStudent()

### Average mark given by Supervisors

In [ ]:
#Plot of the graphs
#-------------------------------------------------
f, ax = plt.subplots(figsize=(15, 4))
sns.set_color_codes('pastel')

#---mean mark
sns.barplot(ax = ax, x = 'Supervisor_ID', y = 'Mean_mark', data = dfProgSup,
            label = '', color = 'b', edgecolor = 'w')
ax.set(ylabel="mean mark", xlabel="Supervisor ID")
ax.set_title('Plot of the average mark given by each supervisor')
sns.despine(left = True, bottom = True) #remove graph frame
plt.show()

### plotStatSupervisor(...)

In [ ]:
def plotStatSupervisor(dfProgSup_s,dfProgSup,dashbord,sort=False,module=""):
    """
    This function plot the statistics for supervisors
    ---> input: 
    - dfProgSup_s = computeProgress(stacked=True)
    - dfProgSup = computeProgress()
    - sort (boolean): if True, the function sort dfProgSup and dfProgSup_s by satisfaction
    - module (string): if the module "mcm" is entered, the fucntion plot the statistics for the module "mcm" only
    ---> output: 
    - graph 1: plot of the proportion of satisfactory/moderate/unsatisfactory progress for each supervisor
             + mean mark given by each supervisor
    - graph 2: plot of the mark dispersion + number of students for each supervisor
    """
    #----- to plot for a particular module
    titleText = ""
    titleText2 = ""
    if module != "":
        dfProgSup = computeModule(dfProgSup,dashbord) #add modules columns
        dfProgSup_s = computeModule(dfProgSup_s,dashbord)#add modules columns
        
        dfProgSup = dfProgSup.iloc[np.where(dfProgSup_m[module] == 1)]#keeps only rows where the module is "module"
        dfProgSup_s = dfProgSup_s.iloc[np.where(dfProgSup_m[module] == 1)]#keeps only rows where the module is "module" 
        titleText = " for module " +  r"$\bf{" + module + "}$" 
    
    #----- Sort values  
    sorter = dfProgSup.Supervisor_ID.values #definition of the list with which to sort values
    if sort == True:
        #sort by satisfaction the dataframes 'dfProgSup_s' and 'dfProgSup'
        dfProgSup_s = dfProgSup_s.sort_values(by=["unsatisfactory","moderate"],ascending = [False,False])
        dfProgSup = dfProgSup.sort_values(by=["unsatisfactory","moderate"],ascending = [False,False])
        sorter = dfProgSup.Supervisor_ID.values #update sorter to also sort the boxplot
        titleText2 = " sorted by " +  r"$\bf{" + "satisfaction" + "}$"  
    
    f, (ax1,ax3) = plt.subplots(2,1,figsize=(15, 10))
    sns.set_color_codes('pastel')
    
    ##################### GRAPH 1 #####################
    #----- 1. Progress per Supervisor
    dfProgSup_s[["unsatisfactory","moderate","satisfactory"]].plot(kind='bar', ax=ax1, stacked=True, 
            color=['red', 'mediumblue', 'g'], alpha = 0.8,edgecolor = 'w',width = 0.8) 
    
    #----- 2. Add mean mark per Supervisor
    ax2 = ax1.twinx()
    xvalues = np.arange(0,len(dfProgSup))#so that there is no shift between the graphs ax1 and ax2
    
    sns.scatterplot(ax = ax2,x = xvalues, y = 'Mean_mark', data = dfProgSup,color = 'slateblue',edgecolor = 'w')
    sns.lineplot(ax = ax2,x = xvalues, y = 'Mean_mark', data = dfProgSup,alpha = 0.5,
                 label = 'mean mark', color = 'slateblue')   
    
    #----- 3. Define axes
    #-- ax1
    ax1.set(xlabel="Supervisor ID", ylabel="proportion of each Progress")
    ax1.set_title('Progress and mean mark for each Supervisor' + titleText + titleText2,pad=20)
    ax1.legend(bbox_to_anchor= (1.05, 1.2))
    #-- ax2
    ax2.set(ylabel="mean mark")
    ax2.tick_params(axis='y', colors="slateblue")
    ax2.yaxis.label.set_color("slateblue")
    ax2.legend(bbox_to_anchor= (1.05, 1.3)) 
    
    #----- 4. Add mean mark value above each point 
    for i, v in enumerate(dfProgSup["Mean_mark"]):
        if np.isnan(v) == False :
            ax2.text(i, v+1.5, "%d" %v, ha="center",color = 'slateblue', size = 10)     
            
    #----- 5. Background parameters        
    sns.despine(left = True, bottom = True)  #remove graph frame
    plt.grid(linestyle = 'dotted')
    
    ##################### GRAPH 2 #####################
    #----- 1. Plot mark dispersion
    flierprops = dict(markerfacecolor='0.75', markersize=4,linestyle='none') #size & color of outliers
    sns.boxplot(ax = ax3,x='Supervisor', y='Marks', data=dashbord,flierprops=flierprops,linewidth=1,order = sorter)
    xlabels = dfProgSup["Supervisor_ID"].values
    
    #----- 2. Add number of students per Supervisor
    ax4 = ax3.twinx()
    sns.scatterplot(ax = ax4, x = xvalues, y = 'nb_students', data = dfProgSup,color = 'red',edgecolor = 'w')
    sns.lineplot(ax = ax4, x = xvalues, y = 'nb_students', data = dfProgSup,alpha = 0.5, color = 'red')
    
    #----- 3. Add number of meetings per Supervisor
    ax5 = ax3.twinx()   
    sns.scatterplot(ax = ax5, x = xvalues, y = 'nb_meetings', data = dfProgSup,color = 'green',edgecolor = 'w')
    sns.lineplot(ax = ax5, x = xvalues, y = 'nb_meetings', data = dfProgSup,alpha = 0.5,color = 'green',linestyle=':')
    
    #----- 4. Define axes
    tkw = dict(size=2, width=1.5)   
    #-- ax3
    ax3.set_xticklabels(xlabels)
    ax3.set(ylabel="mark dispersion", xlabel="Supervisor ID")
    ax3.set_title('Mark dispersion and number of students for each Supervisor' + titleText + titleText2)    
    ax3.tick_params(axis='x', which='major',rotation=90)
    #-- ax4
    ax4.set(ylabel="number of students")
    ax4.yaxis.label.set_color("red")
    ax4.tick_params(axis='y', colors="red", **tkw)
    #-- ax5
    ax5.spines.right.set_position(("axes", 1.04)) #offset the right spine of ax5 
    ax5.set(ylabel="number of meetings")
    ax5.yaxis.label.set_color("green")
    ax5.tick_params(axis='y', colors="green", **tkw)
    #-- legend
    p1, = ax4.plot([0,0,0], color="red",linestyle = '-', label="number of students");p1.remove()
    p2, = ax5.plot([0,0,0], color = "green",linestyle = ':', label="number of meetings");p2.remove() 
    ax3.legend(handles=[p1, p2],bbox_to_anchor= (1.05, 1.2))    
    
    #----- 5. Background parameters 
    plt.subplots_adjust(hspace=0.4) #set space between the 2 plots
    sns.despine(left = True, bottom = True) #remove graph frame

In [ ]:
#plotStatSupervisor(dfProgSup_s,dfProgSup,dashbord)
plotStatSupervisor(dfProgSup_s,dfProgSup,dashbord,sort = True)

<font color = "red"> Some supervisors have 0 students because there is only NaN students IDs in the dataset for the given supervisor.

In [ ]:
plotStatSupervisor(dfProgSup_s,dfProgSup,dashbord,module="mcm")
plotStatSupervisor(dfProgSup_s,dfProgSup,dashbord,module="ca326")
plotStatSupervisor(dfProgSup_s,dfProgSup,dashbord,module="ca400")
plotStatSupervisor(dfProgSup_s,dfProgSup,dashbord,module="ca472")
plotStatSupervisor(dfProgSup_s,dfProgSup,dashbord,module="pnu")

### plotWhoFilledDur(...)

In [ ]:
def plotWhoFilledDur(dfProgSup):
    """
    ---> input: dfProgSup = computeProgress() 
    only the columns 'Supervisor_ID' and '%FilledDuration' are used
    ---> output: 
    - Barplot of the percentage of meetings filled with a duration for each supervisor
    - Lineplot of the number of meetings for each supervisor
    """
    
    f, ax = plt.subplots(figsize=(20, 5))
    sns.set_color_codes('pastel')

    #----- 1. Barplot of the percentage of meetings with filled duration for each supervisor
    sns.barplot(ax = ax, x = 'Supervisor_ID', y = '%FilledDuration', data = dfProgSup,
                label = '', color = 'b', edgecolor = 'w')
    #----- 2. Plot of the number of meetings for each supervisor
    xvalues = np.arange(0,len(dfProgSup))
    ax2 = ax.twinx()   
    sns.scatterplot(ax = ax2, x = xvalues, y = 'nb_meetings', data = dfProgSup,color = 'green',edgecolor = 'w')
    sns.lineplot(ax = ax2, x = xvalues, y = 'nb_meetings',
                 data = dfProgSup,alpha = 0.5,color = 'green',linestyle = ':')  

    #----- 4. Definition of axes and background
    #--ax
    ax.set_title('Plot of the % of meetings with a filled duration and the number of meetings for each supervisor', pad=30)
    ax.set(ylabel="% of meetings with a filled duration", xlabel="Supervisor ID")
    ax.tick_params(axis='y', colors="slateblue")
    ax.yaxis.label.set_color("slateblue")
    #--add percenatage value above each bar in the barplot
    for i, v in enumerate(dfProgSup['%FilledDuration']):
            ax.text(i, v+2, v, ha="center",color = 'slateblue', size = 10, rotation = 90)  
    #--ax2
    ax2.set(ylabel="number of meetings")
    ax2.tick_params(axis='y', colors="green")
    ax2.yaxis.label.set_color("green")
    #--background color
    ax.patch.set_facecolor('grey')
    ax.patch.set_alpha(0.05)

    sns.despine(left = True, bottom = True) #remove graph frame
    plt.show()

In [ ]:
plotWhoFilledDur(dfProgSup)

In [ ]:
ID_durMoreThan,ID_durLessThan = retrieveWhoFilledDur(dfProgSup,durMoreThan = 80,
                                                     durLessThan = 20, display = True)

<font color = "green" > __To do__:  statistic significance tests of the mark between one year to the other. Paired t-test to see if the module have the same mean. \
<font color = "green" > __To do__:  try to fit multivariate distrib \
<font color = "green" > __To do__:  bin the marks (5 per 5) and by the marking system (see picture) 

<font color = "red" > __To do__: transform mean into median for most plots

## Statistical Normality Tests

__Statistical Test for Normality of variable "Marks":__ \
$\mathcal{H}_0: $ variable Marks follows a Gaussian distribution.\
$\mathcal{H}_1: $ variable Marks does not follows a Gaussian distribution. \
If the p-value of the test is less than __0.05__ (5%), we __reject ${H}_0$__ and conclude that Marks doesn't follow a Gaussian distriution.

__Test 1: Kolmogorov-Smirnov__ \
Kolmogorov-Smirnov is a statistical test for goodness of fit. This test compares two distributions (in this case, one of the two distributions is Gaussian). The null hypothesis for this test is, both the distributions are identical (or) there is no difference between the two distributions. Kolmogorov-Smirnov test expects the input variable to have a perfect normal distribution.

__Test 2: Shapiro-Wilk__ \
A disadvantage of the Shapiro-Wilk test is that it is unreliable as soon as the sample size (or) the length of a variable exceeds 5000.

__Test 3: D’Agostino and Pearson__ \
This test doesn’t expect the distribution to be perfectly normal, but near to normal.

In [ ]:
print("*********************************************************")
print("Gaussian Tests for variable Marks")
print("*********************************************************")
print("size of n0Marks = ",len(n0Marks))
print("")
print("--- Kolmogorov-Smirnov test:"); KS = stats.kstest(n0Marks, 'norm')
print(f'{"Not Gaussian: " if KS[1]<0.05 else "Gaussian: "} {KS}')
print("")
print("--- Shapiro-Wilk test:"); SW = stats.shapiro(n0Marks)
print(f'{"Not Gaussian: " if SW[1]<0.05 else "Gaussian: "} {SW}')
print("")
print("--- D’Agostino and Pearson’s test:"); AP = stats.normaltest(n0Marks)
print(f'{"Not Gaussian: " if AP[1]<0.05 else "Gaussian: "} {AP}')
print("")

In [ ]:
print("*********************************************************")
print("Gaussian Tests for variable Marks for each module")
print("*********************************************************")
for i in range(len(modulesList)):
    print("Module "+modulesList[i]+":")
    print("---------------------------------------------------------")
    print("--- Kolmogorov-Smirnov test:"); KS = stats.kstest(marksPerMod[i], 'norm')
    print(f'{"Not Gaussian: " if KS[1]<0.05 else "Gaussian: "} {KS}')
    print("")
    #print("--- Shapiro-Wilk test:"); SW = stats.shapiro(marksPerMod[i])
    #print(f'{"Not Gaussian: " if SW[1]<0.05 else "Gaussian: "} {SW}')
    #print("")
    #print("--- D’Agostino and Pearson’s test:"); AP = stats.normaltest(marksPerMod[i])
    #print(f'{"Not Gaussian: " if AP[1]<0.05 else "Gaussian: "} {AP}')
    #print("")

## Export reorganised dahbord

This can be useful to perform anova/ancova tests

In [ ]:
def Reorganise(dashbord):
    """
    This function takes the dashbord as input and reorganises the information into a new dashbord.
    input: 
    - dashbord : original dashbord data
    output: 
    - df : new dashbord data where: 
        column "approved" has been deleted
        rows where the type correponds to "ethics" have been deleted
        nbSatis = number of progress satisfactory for the student
        nbModer = number of progress moderate for the student
        nbUnsat = number of progress unsatisfactory for the student
        startDate = first meeting or approval
        endDate = last meeting or approval
        totDuration = total meeting duration for the student
        Year = gradutation year of the student
        Marks = mark of the student
        Supervisor = supervisor if the student
        module = module of the student
    """
    dashbord = dashbord.drop(columns='approved',axis=1) #remove column "approved"
    #drop rows corresponding to type "ethics":
    dashbord = dashbord.drop(index=dashbord.iloc[np.where(dashbord["type"]=='ethics')].index.tolist())
    #compute stuIndex = Indices (row labels in the dataframe i.e. use .loc[stuIndex]) 
    #of students who received a grade. Cannot use stuIndex from previous function
    #since the dashbord indexes have changed with .drop()
    stuIndex = dashbord["Student"].drop_duplicates().index 
    nanStuIndex = np.where(dashbord["Student"].drop_duplicates().isna() == True) #find index of NaN values
    stuIndex = np.delete(stuIndex,nanStuIndex) #remove the NaN value from StuIndex
    df = pd.DataFrame(dashbord[["module","Year","Student","Supervisor","Marks"]].loc[stuIndex])
    
    studentList = dashbord["Student"].loc[stuIndex].values
    startDate = []
    endDate = []
    duration = []
    nbSatis = []
    nbModer = []
    nbUnsat = []

    for s in studentList:
        idx = np.where(dashbord["Student"]==s)
        startDate.append(dashbord["date"].iloc[idx].min())
        endDate.append(dashbord["date"].iloc[idx].max())
        duration.append(dashbord["duration"].iloc[idx].sum())
        nbSatis.append(dict(Counter(dashbord["progress"].iloc[idx])).get('satisfactory'))
        nbModer.append(dict(Counter(dashbord["progress"].iloc[idx])).get('moderate'))
        nbUnsat.append(dict(Counter(dashbord["progress"].iloc[idx])).get('unsatisfactory'))

    nbSatis = np.array(nbSatis); nbSatis[np.where(nbSatis == None)[0]] = 0
    nbModer = np.array(nbModer); nbModer[np.where(nbModer == None)[0]] = 0
    nbUnsat = np.array(nbUnsat); nbUnsat[np.where(nbUnsat == None)[0]] = 0
    
    df["nbSatis"] = nbSatis
    df["nbModer"] = nbModer
    df["nbUnsat"] = nbUnsat
    df["startDate"] = startDate
    df["endDate"] = endDate
    df["totDuration"] = duration

    dfn0 = df.copy() #dataFrame where Marks = 0 have been removed
    idx0Marks = np.where(df["Marks"]==0) #indexes where the Marks = 0
    dfn0["Marks"] = dfn0["Marks"].drop(index=df["Marks"].iloc[idx0Marks[0]].index.tolist())
    return(df,dfn0)

In [ ]:
#dashbordR, dashbordR0 = Reorganise(dashbord)

In [ ]:
#dashbordR.to_csv('dashbordR.csv')
#dashbordR0.to_csv('dashbordR0.csv')

In [ ]:
#use ANOVA even if variable is not gaussian:
#https://stats.stackexchange.com/questions/5680/can-i-trust-anova-results-for-a-non-normally-distributed-dv

__Question:__
For the rows where there is no student nor mark, there is a Supervisor number. But I cannot know if the supervisor number correspondonds to 1 student or to several students? Can I consider that it corresponds to 1 student?\
I need this info to know for example the number of student per Supervisor. 

## Interactive dashbords

In [ ]:
#libraries for interactive dashbords
import holoviews as hv
import param
import panel as pn
#from panel.pane import LaTeX
#hv.extension('bokeh')
#hv.extension('matplotlib')
pn.extension()
from PIL import Image
from io import BytesIO
import requests
import tkinter #otherwise put error for plt.show()

In [ ]:
#dashbord documentation:
#https://coderzcolumn.com/tutorials/data-science/how-to-create-dashboard-using-python-matplotlib-panel
#documentation
#https://panel.holoviz.org/user_guide/Widgets.html
#https://www.quansight.com/post/building-a-panel-pipeline

### StudentProgress_Dashbord

In [ ]:
class StudentProgress_Dashbord(param.Parameterized):
    #Min = int(dashbord["Student"].min()) #if we sort by student_ID
    #Max = int(dashbord["Student"].max()) #if we sort by student_ID
    Select_begin_index = param.Integer(0,bounds=(0,None))
    Select_end_index = param.Integer(200,bounds=(0,None))
    Sort_by = param.ObjectSelector(default=["Marks"], objects=[["Student_IDs"], ['satisfactory','moderate'], ["Marks"]])
    Select_font_size = param.Number(6,bounds=(1,8))
    Select_point_size = param.Integer(15, bounds=(0,30))
    def view(self):       
        dfProgStu_s = computeProgress("Student",stacked = True)
        dfProgStu_s["Student_IDs"] = dfProgStu_s.index
        dfProgStu_s = dfProgStu_s.sort_values(by=self.Sort_by)
    
        #begin = np.where(dfProgStu_s[self.Sort_by] == self.Select_begin)[0][0] #if we sort by student_ID
        #end = np.where(dfProgStu_s[self.Sort_by] == self.Select_end)[0][0]+1 #if we sort by student_ID
        begin = self.Select_begin_index
        end = self.Select_end_index
        
        #---progress per Student
        fontS = 6
        fontL = 8
        font = self.Select_font_size
        f, ax= plt.subplots(figsize=(7, 3),constrained_layout=True)
        dfProgStu_s[["unsatisfactory","moderate","satisfactory"]].iloc[begin:end].plot(kind='bar', ax=ax, stacked=True, 
        color=['red', 'mediumblue', 'g'], alpha = 0.8,edgecolor = 'w',width = 0.8)      
        ax.set_title('Progress and mark for each Student')
        ax.set_xlabel("Student ID",fontsize=fontS)
        ax.set_ylabel("Progress",fontsize=fontS)
        ax.tick_params(axis='x', which='major',rotation=90)
        ax.tick_params(axis='both', which='major', labelsize=font)     
        
        #--- add student mark
        ax2 = ax.twinx()
        #so that there is no shift between the graphs ax1 and ax2
        Marks = dfProgStu_s["Marks"].values
        ax2_xval = np.arange(0,len(Marks[begin:end]),1)
        
        #add mean mark value above each point 
        for i, v in enumerate(Marks[begin:end]):
            if np.isnan(v) == False :
                ax2.text(i, v+1.5, "%d" %v, ha="center",color = 'slateblue', size = font)
        
        sns.scatterplot(ax = ax2,x = ax2_xval, y = Marks[begin:end], color = 'slateblue',
                        edgecolor = 'w',s = self.Select_point_size)
        
        sns.lineplot(ax = ax2,x = ax2_xval, y = Marks[begin:end], alpha = 0.5, linewidth  = 1,
                 label = 'student mark', color = 'slateblue')
        ax2.set_ylabel("mark",fontsize=fontS)
        ax2.tick_params(axis='both', which='major', labelsize=font)
        
        ax.set_title('Progress and mark for each Student',fontsize=fontL)
        sns.despine(left = True, bottom = True)  #remove graph frame
        ax.legend(bbox_to_anchor= (1.0, 1.25),fontsize=fontS)
        ax2.legend(bbox_to_anchor= (1.0, 1.35),fontsize=fontS)
        plt.grid(linestyle = 'dotted')
        plt.close(f)
        return(f)

In [ ]:
dash0=StudentProgress_Dashbord()
pn.Column(dash0.param,dash0.view)

### GlobalStudentProgress_Dashbord

In [ ]:
class GlobalStudentProgress_Dashbord(param.Parameterized):
    Sort_by = param.ObjectSelector(default=["Marks"], objects=[["Marks"],["unsatisfactory"],["satisfactory"],["moderate"]])
    Select_to_show = param.ObjectSelector(default=["unsatisfactory"], objects=[["unsatisfactory"],["moderate"],["satisfactory"]])
    Mask_value =  param.ObjectSelector(default=["-1"], objects=[["-1"],["0"],["1"]])
    Point_size_1 = param.Integer(10, bounds=(0,15)) 
    Point_size_2 = param.Integer(2, bounds=(0,15)) 
    
    def view(self): 
        #--- values definition
        #customisable parameters
        to_show = self.Select_to_show[0]
        mask_value = self.Mask_value[0]
        s1 = self.Point_size_1
        s2 = self.Point_size_2 
        #font size
        fontS = 6
        fontL = 8
        #dataframe and tables 
        dfProgStu = computeProgress("Student")
        dfProgStu = dfProgStu.sort_values(by=self.Sort_by)
        Marks = dfProgStu["Marks"].values
        ax2_xval = np.arange(0,len(Marks),1)
        #dictionaries for plots
        show = np.full(len(dfProgStu[to_show]),"             ")
        show[np.where(dfProgStu[to_show]!=int(mask_value))]="non {} values".format(mask_value)
        P = {"unsatisfactory":["w", "r"],"satisfactory":["g", "w"],"moderate":["mediumblue","w"]}
        C = {"unsatisfactory":"r","satisfactory":"g","moderate":"mediumblue"}
        S = {"-1":None,"0": show,"1":show}
       
        #--- plots
        f, ax= plt.subplots(figsize=(7, 3),constrained_layout=True)
        
        #plot of progress
        sns.scatterplot(ax=ax, x = ax2_xval, y = dfProgStu[to_show].values, alpha = 1, s  = s1,
                 label = '{}'.format(to_show),color = C[to_show], hue = S[mask_value], palette=P[to_show])
        #labels and titles
        ax.set_title('Progress and mark for each Student sorted by variable "{}"'.format(self.Sort_by[0]),fontsize=fontL)
        ax.set_xlabel("indexes in the dataframe corresponding to a student",fontsize=fontS)
        ax.set_ylabel("Proportion of {} for each student".format(to_show),fontsize=fontS)
        ax.tick_params(axis='x', which='major',rotation=90)
        ax.tick_params(axis='both', which='major', labelsize=fontS)    
        ax.legend(bbox_to_anchor= (1.0, 1.16),fontsize=fontS)
        #plot of Marks
        ax2 = ax.twinx()
        sns.scatterplot(ax=ax2, x = ax2_xval, y = Marks, alpha = 1, s  = s2, label = 'student mark', color = 'black')
        #labels and titles
        ax2.set_ylabel("mark",fontsize=fontS)
        ax2.tick_params(axis='both', which='major', labelsize=fontS)
        ax2.legend(bbox_to_anchor= (1.0, 1.25),fontsize=fontS)
        sns.despine(left = True, bottom = True)  #remove graph frame
        plt.grid(linestyle="dotted")
        plt.close(f)
        return(f)

In [ ]:
dash3=GlobalStudentProgress_Dashbord()
pn.Column(dash3.param,dash3.view)

### stats1Student_Dashbord

In [ ]:
def stats1Student(studID):
    """
    This function gives the statistics of a student and the temporal evolution of his/her progress
    """
    #*********************************************************************
    # Computation of values
    #*********************************************************************
    #--- indexes definition
    idx = np.where(dashbord["Student"]==studID) #index (no row names i.e. use .iloc[idx]) associated with student 
    stuMark = dashbord["Marks"].iloc[idx[0][0]].astype(int)
    dates = dashbord["date"].iloc[idx].sort_values()#sort the dates by ascending order
    #do not use idx after this since it is not sorted by time. Use the dates.index instead
    
    #--- initialisation
    #These tabs values are in the same order as the dates (ascending order):
    meetTab0 = np.zeros(len(dates),dtype=int) #duration of the meeting 
    meetTab1 = np.zeros(len(dates),dtype=str) #progress of the meeting 
    meetTab2 = np.zeros(len(dates),dtype=str) #type of the meeting: consider only approval or meeting and no ethics
    
    #--- value assignment
    #np.nan == np.nan returns False. We have to define a nan for the strings:
    nan_str = np.array([np.nan]).astype(str)[0]
    
    meetTab0 = dashbord["duration"].loc[dates.index].to_numpy() #ok you have to use .loc[]
    meetTab1 = dashbord["progress"].loc[dates.index].to_numpy()
    meetTab2 = dashbord["type"].loc[dates.index].to_numpy()
    
    idxEthics = np.where(meetTab2=='ethics')#gets the index where there is the type 'ethics'.
    
    nbMeetin = len(np.where(meetTab2=='meeting')[0])
    nbApprov = len(np.where(meetTab2=='approval')[0])
    nbEthics = len(np.where(meetTab2=='ethics')[0])
    
    #--- delete the column corresponding to 'ethics' in the 3 tabs and in the dates
    meetTab0 = np.delete(meetTab0,idxEthics)
    meetTab1 = np.delete(meetTab1,idxEthics)
    meetTab2 = np.delete(meetTab2,idxEthics) 
    dates = dates.drop(labels=dates.index[idxEthics])

    #--- deals with the nan value in string arrays
    nanTab1 = np.where(meetTab1.astype(str)==nan_str)
    meetTab1[nanTab1]='unkown' #'unkown' if no progress found otherwise the 3 np.where below don't work
    nanTab2 = np.where(meetTab2.astype(str)==nan_str)
    meetTab2[nanTab2]='unkown'
    #see: https://stackoverflow.com/questions/46055853/cant-find-nan-entries-using-numpy-in-array-of-strings

    #--- transfrom meetTab1 elements in float values to be able to plot it.
    Satif = np.where(meetTab1=='satisfactory')
    Moder = np.where(meetTab1=='moderate')
    Unsat = np.where(meetTab1=='unsatisfactory')
    #if the duration array is not empty, change de meetTab0 value so it can be 
    #plotted on the same graph as the duration:
    if not(np.all(np.isnan(meetTab0))) and not(np.all(meetTab0 == 0)):
        minT = np.nanmin(meetTab0)#nanmax ignore nan values
        maxT = np.nanmax(meetTab0)
        medT = (maxT+minT)/2
        if minT == maxT:
            medT = maxT/2
            minT = maxT/3      
        meetTab1[Satif] = maxT 
        meetTab1[Moder] = medT
        meetTab1[Unsat] = minT
        meetTab1[nanTab1] = np.nan #the value is reset to nan
    else: #otherwise, put a fixed value
        meetTab1[Satif] = 3
        meetTab1[Moder] = 2
        meetTab1[Unsat] = 1
        meetTab1[nanTab1] = np.nan #the value is reset to nan

    #--- transforms 'approval' and 'meeting' values into 'M' and 'A' for better visibility when plotting
    meetTab2[np.where(meetTab2=='approval')] = 'A'
    meetTab2[np.where(meetTab2=='meeting')] = 'M'

    #--- associate a color with the progres in the barplot
    barColors = np.full(len(meetTab1),'w') #to plot the color of the barplot
    barColors[Satif] = 'g'
    barColors[Moder] = 'b'
    barColors[Unsat] = 'r'
    
    return dates,meetTab0,meetTab1,meetTab2,barColors,nbMeetin,nbApprov,nbEthics,stuMark

In [ ]:
class stats1Student_Dashbord(param.Parameterized):
    #student_ID = param.Integer(72,bounds=(int(dashbord["Student"].min()),int(dashbord["Student"].max())))     
    student_ID = param.Integer(1,bounds=(1,None))
    def view(self):
        studID = self.student_ID
        dates,meetTab0,meetTab1,meetTab2,barColors,nbMeetin,nbApprov,nbEthics,stuMark=stats1Student(studID)
        f,ax = plt.subplots(figsize=(7, 3),sharex=True,constrained_layout=True)
        fontS = 6
        fontL = 8
        #--- main plots   
        ax.bar(x=dates, height=meetTab1,width=2.0, align='center',
                edgecolor = 'w',alpha=0.5,zorder=1,color=barColors)
        ax.scatter(x=dates,y=meetTab0,label='duration (min)',edgecolor = 'w', zorder=2, s=30)

        #--- x-axis of axis 1 (for dates displaying) customization
        ax.set_xlabel("\ndate",fontsize=fontS)
        ax.set_ylabel("duration (min)",fontsize=fontS)
        ax.set_xticks(dates)
        #plt.xticks(rotation = 90)   
        xlabels = []
        for d in dates:
            xlabels.append(d.strftime('%Y-%m-%d'))     
        ax.set_xticklabels(labels=xlabels, rotation=90,fontsize=fontS)
        ax.tick_params(axis='both', which='major', labelsize=fontS)
        ax.spines['top'].set_visible(False) 
        ax.spines['right'].set_visible(False)

        #--- x-axis of axis 2 (for approval or meeting) customization
        ax2 = ax.twiny()
        ax2.set_xticks(ax.get_xticks())
        ax2.set_xbound(ax.get_xbound())
        ax2.set_xticklabels([x for x in meetTab2],fontsize=fontS)
        ax2.spines['top'].set_visible(False)
        ax2.spines['right'].set_visible(False)

        #--- background color
        ax.patch.set_facecolor('grey')
        ax.patch.set_alpha(0.05)

        #--- legend
        handles, labels = ax.get_legend_handles_labels()
        green_patch = mpatches.Patch(color='green', label='progress satisfactory',alpha=0.5)
        blue_patch = mpatches.Patch(color='blue', label='progress moderate',alpha=0.5)
        red_patch = mpatches.Patch(color='red', label='progress unsatisfactory',alpha=0.5)
        M_patch = mpatches.Patch(color='white', label='M: meeting')
        A_patch = mpatches.Patch(color='white', label='A: approval')
        handles.append(green_patch)
        handles.append(blue_patch)
        handles.append(red_patch)
        handles.append(M_patch)
        handles.append(A_patch)
        ax.legend(title= "Legend:\n",title_fontsize = fontL, handles=handles,bbox_to_anchor= (1.02, 1.2),fontsize=fontS)
        ax.set_title("Progress of student n°{}\n".format(studID),fontsize=fontL)
        f.text(1.06, 0.25, 'Mark: {}/100'.format(stuMark), transform=ax.transAxes,fontsize=fontL)
        f.text(1.06, 0.10, '{} meetings'.format(nbMeetin), transform=ax.transAxes,fontsize=fontS)
        f.text(1.06, 0.05, '{} approvals'.format(nbApprov), transform=ax.transAxes,fontsize=fontS)
        f.text(1.06, 0.0, '{} ethics'.format(nbEthics), transform=ax.transAxes,fontsize=fontS)
        plt.close(f)
        return(f)

In [ ]:
dash1=stats1Student_Dashbord()
pn.Column(dash1.param,dash1.view)

### stats1Supervisor_Dashbord

In [ ]:
def stats1Supervisor(supID,groupBy,date1,date2): 
    if groupBy == "Days":
        groupBy = 'D'
    if groupBy == "Months":
         groupBy = 'M'
    if groupBy == "Years":
         groupBy = 'Y'
    
    #*********************************************************************
    # Computation of values
    #*********************************************************************
    idx = np.where(dashbord["Supervisor"]==supID) #index (no row names i.e. use .iloc[idx]) associated with supervisor 
    #do not use idx after since it is not sorted by time. Use the dates.index instead
    dates = dashbord["date"].iloc[idx].sort_values()#sort the dates by ascending order
    datesU = dates.unique()

    countType = np.zeros((3,(len(dates.unique())))) #counts the number of 'meeting', 'approval' and 'ethics' 
                                                    #for each unique date i.e. for each day
    #columns: the date of the day by cending order (dates are unique)
    #row0: number of 'meetings' for a given date
    #row1: number of 'approval' for a given date
    #row2: number of 'ethics' for a given date
    durTab = np.zeros(len(datesU))

    i=0
    for d in dates.unique():
        idxToday = np.where(dates==d)
        count = Counter(dashbord["type"].loc[dates.index].iloc[idxToday])
        count = dict(count) #transform object Counter to dict
        if 'meeting' in count.keys():     
            countType[0,i] = count.get('meeting') 
        if 'approval' in count.keys():     
            countType[1,i] = count.get('approval')
        if 'ethics' in count.keys():     
            countType[2,i] = count.get('ethics')

        Dur = dashbord["duration"].loc[dates.index].iloc[idxToday]
        for du in Dur:            
            if not (np.isnan(du)):
                durTab[i] += du  
        i+=1

    #************************************************************
    # date calibration
    #************************************************************
    #creation of a dataFrame
    df = pd.DataFrame(datesU)
    df = df.rename(columns={0: "date"}) 
    df["nb_meetings"] = countType[0,:].astype(int)
    df["nb_approvals"] = countType[1,:].astype(int)
    df["nb_ethics"] = countType[2,:].astype(int)
    df["duration"] = durTab.astype(int)

    #definition start date & end date for the given supervisor.
    startDate = df.date.min(skipna=True)
    endDate = df.date.max(skipna=True)

    #group per day (does nothing) month or year
    per = df["date"].dt.to_period(groupBy) 
    g = df.groupby(per)
    g = g.sum()
    g.index = g.index.to_timestamp()
    g.reset_index(inplace=True)

    #create a DataFrame with the dates between the 2 values date1 and date2
    newdf = g[g['date'].between(date1, date2)]

    return newdf

In [ ]:
class stats1Supervisor_Dashbord(param.Parameterized):
    supervisor_ID = param.Integer(int(dashbord["Supervisor"].min()), bounds=(int(dashbord["Supervisor"].min()),int(dashbord["Supervisor"].max())))
    Group_by = param.ObjectSelector(default="Months", objects=["Days", "Months", "Years"])
    Start_date = param.Date(dashbord["date"].min(),bounds=(dashbord["date"].min(), dashbord["date"].max()))
    End_date = param.Date(dashbord["date"].max(),bounds=(dashbord["date"].min(),dashbord["date"].max()))
    Bar_width = param.Number(3, bounds=(0,20))
    Point_size = param.Integer(20, bounds=(0,100))
    
    def view(self):
        newdf=stats1Supervisor(self.supervisor_ID,self.Group_by,self.Start_date,self. End_date)
        f,ax = plt.subplots(figsize=(7, 3),sharex=True,constrained_layout=True)
        fontS = 6
        fontL = 8
        ax2 = ax.twinx()

        #--- main plots   
        width = self.Bar_width
        ax.bar(x=newdf.date, height=newdf.nb_meetings, width=width, align='center',
                zorder=1,edgecolor = 'w',color='forestgreen',alpha=0.5)
        ax.bar(x=newdf.date, height=newdf.nb_approvals, width=-width, align='edge',
                zorder=2,edgecolor = 'w',color='slateblue',alpha=0.5)
        ax.bar(x=newdf.date, height=newdf.nb_ethics, width=width, align='edge',
                zorder=3,edgecolor = 'w',color='orangered',alpha=0.5)
        ax2.scatter(x=newdf.date, y=newdf.duration,s = self.Point_size,edgecolor = 'w',
                    color='slateblue',label = 'duration',zorder=4)

        #--- axis customization
        ax.set_ylabel("occurence",fontsize=fontS)
        ax2.set_ylabel("duration(min)",fontsize=fontS)
        ax2.set_xlabel("\ndate",fontsize=fontS)
        ax2.set_xticks(newdf.date)
        ax.yaxis.set_major_locator(MaxNLocator(integer=True)) #integer values for yaxis   
        ax2.yaxis.set_major_locator(MaxNLocator(integer=True)) #integer values for yaxis   
        ax2.set_ylim(bottom=0.)
        ax.set_ylim(bottom=0.)
        ax.tick_params(axis='both',which='major',labelsize=fontS)
        ax2.tick_params(axis='both',which='major',labelsize=fontS)

        xlabels = []
        for d in newdf.date:
            xlabels.append(d.strftime('%Y-%m-%d'))     
        ax.set_xticklabels(labels=xlabels, rotation=90)

        #--- remove frame
        ax.spines['top'].set_visible(False) 
        ax.spines['right'].set_visible(False)
        ax2.spines['top'].set_visible(False) 
        ax2.spines['right'].set_visible(False)

        #--- background color
        ax.patch.set_facecolor('grey')
        ax.patch.set_alpha(0.05)
        ax2.patch.set_facecolor('grey')
        ax2.patch.set_alpha(0.05)

        #--- legend
        handles, labels = ax.get_legend_handles_labels()
        green_patch = mpatches.Patch(color='forestgreen', label='number of meetings',alpha=0.5)
        blue_patch = mpatches.Patch(color='slateblue', label='number of approvals',alpha=0.5)
        red_patch = mpatches.Patch(color='orangered', label='number of ethics',alpha=0.5)
        handles.append(green_patch)
        handles.append(blue_patch)
        handles.append(red_patch)
        ax.legend(title= "Legend:\n",title_fontsize = fontL, handles=handles,bbox_to_anchor= (1.35, 1),fontsize=fontS)
        ax.set_title("Statistics of Supervisor n°{}\n".format(self.supervisor_ID),fontsize=fontL)
        f.text(1.1, 0.5, 'For the selected period:', transform=ax.transAxes,fontsize=fontS)
        f.text(1.1, 0.45, '- total duration: {} min'.format(newdf["duration"].sum()), transform=ax.transAxes,fontsize=fontS)
        f.text(1.1, 0.40, '  or {} hours'.format(round(newdf["duration"].sum()/60,2)), transform=ax.transAxes,fontsize=fontS)
        f.text(1.1, 0.35, '- {} meetings'.format(newdf["nb_meetings"].sum()), transform=ax.transAxes,fontsize=fontS)
        f.text(1.1, 0.30, '- {} approvals'.format(newdf["nb_approvals"].sum()), transform=ax.transAxes,fontsize=fontS)
        f.text(1.1, 0.25, '- {} ethics'.format(newdf["nb_ethics"].sum()), transform=ax.transAxes,fontsize=fontS)
        plt.grid(linestyle = 'dotted')
        plt.close(f)
        return(f)

In [ ]:
dash2=stats1Supervisor_Dashbord()
pn.Column(dash2,dash2.view)

### Other plots

Plot of the average mark per year: 

In [ ]:
#Computation of the mean mark per year
#-------------------------------------------------
years = dashbord["Year"].drop_duplicates().values
meanYear = np.zeros((2,len(years))) #mean mark per Year. Row 0: Year, Row 1: mark
meanYear[0,:] = years #Row 0: Year
YearDisp = []#list containing the list of grades for each year

i = 0
for y in years:
    idx = np.where(dashbord["Year"].loc[stuIndex] == y)
    meanYear[1,i] =  mean(dashbord["Marks"].loc[stuIndex].iloc[idx])
    YearDisp.append(dashbord["Marks"].loc[stuIndex].iloc[idx].values)
    i+=1

#Data fromatting
#-------------------------------------------------
dfmeanYear = pd.DataFrame(meanYear.T) #transformation to dataframe to plot graph with seaborn
dfmeanYear.columns = {'Year','Mean_mark'}
dfmeanYear['Year'] = dfmeanYear['Year'].astype(int)

dfYearDisp = pd.DataFrame(YearDisp) #transformation to dataframe to plot graph with seaborn
dfYearDisp.index = years.astype(int)
dfYearDisp = dfYearDisp.sort_index().T
#the columns names of the dataframe correspond to the year
#the row indices of the dataframe correspond to the number of grades per year

In [ ]:
#Plot of the graph
#-------------------------------------------------
f, (ax1,ax2) = plt.subplots(1,2, figsize=(10,5))
sns.set_color_codes('pastel')

#--- mean mark
sns.barplot(ax=ax1, x = 'Year', y = 'Mean_mark', data = dfmeanYear,
            label = 'Total', color = 'b', edgecolor = 'w')
ax1.set(ylabel="mean mark", xlabel="Year")
ax1.set_title('Plot of the average mark per year')

#--- mark dispersion
flierprops = dict(markerfacecolor='0.75', markersize=4,linestyle='none') #size & color of outliers
sns.boxplot(ax=ax2, data = dfYearDisp,flierprops=flierprops,linewidth=1)
ax2.set(ylabel="mark dispersion", xlabel="Year")
ax2.set_title('Plot of the mark dispersion per year')

sns.despine(left = True, bottom = True)
plt.show()

Plot of the average mark per subject:

In [ ]:
#Computation of the mean mark per module
#-------------------------------------------------
modules = dashbord["module"].cat.categories
meanModules= [] #mean mark per module!
moduleDisp = []#list containing the list of grades for each module

for m in modules:
    idx = np.where(dashbord["module"].loc[stuIndex] == m)
    meanModules.append(mean(dashbord["Marks"].loc[stuIndex].iloc[idx]))
    moduleDisp.append(dashbord["Marks"].loc[stuIndex].iloc[idx].values)

#Data fromatting
#-------------------------------------------------
dfmeanModules = pd.DataFrame(meanModules) #transformation to dataframe to plot graph with seaborn
dfmeanModules.insert(0, "module",modules, True)
dfmeanModules = dfmeanModules.rename(columns={0: "Mean_Mark"})

dfModuleDisp = pd.DataFrame(moduleDisp) #transformation to dataframe to plot graph with seaborn
dfModuleDisp.index = modules
dfModuleDisp = dfModuleDisp.sort_index().T
#the columns names of the dataframe correspond to the module
#the row indices of the dataframe correspond to the number of grades per module

In [ ]:
#Plot of the graph
#-------------------------------------------------
f, (ax1,ax2) = plt.subplots(1,2, figsize=(10,5))
sns.set_color_codes('pastel')

#--- mean mark
sns.barplot(ax=ax1, x = 'module', y = 'Mean_Mark', data = dfmeanModules,
            label = 'Total', color = 'b', edgecolor = 'w')

ax1.set(ylabel="mean mark", xlabel="module")
ax1.set_title('Plot of the average mark per module')

#--- mark dispersion
flierprops = dict(markerfacecolor='0.75', markersize=4,linestyle='none') #size & color of outliers
sns.boxplot(ax=ax2, data = dfModuleDisp,flierprops=flierprops,linewidth=1)
ax2.set(ylabel="mark dispersion", xlabel="module")
ax2.set_title('Plot of the mark dispersion per module')

sns.despine(left = True, bottom = True)
plt.show()

## Multiple Correspondance Analysis

For multiple correspondance analysis, we need only categorical variables. So we transform the quantitative variable Marks in a categorical variable.

MCA documentation: https://notebook.community/wikistat/Exploration/TutosRudim/Cal4-Python-AFCM

In [ ]:
#define a new dataframe
#----------------------------------------------------
dashMCA = dashbord[["Marks","progress","module","type"]]
#remove rows corresponding to type ethics since there is no progress for this row:
dashMCA = dashMCA.drop(index=np.where(dashMCA["type"] == "ethics")[0],axis=0)

#imputation of missing data
#this first imputation is naive but can be improved later
#----------------------------------------------------
dashMCA["Marks"]=dashMCA["Marks"].fillna(dashMCA["Marks"].median()) #naive imputation: imputation with median 
dashMCA["progress"]=dashMCA["progress"].fillna("moderate") #naive imputation: imputation with majority class
dashMCA["type"]=dashMCA["type"].fillna("approval") #naive imputation: imputation with majority class 

#Discretize quantitative variables
#----------------------------------------------------
dashMCA["MarksQ"],bins_edges=pd.qcut(dashMCA["Marks"],3,labels=["low","medium","high"],retbins=True) #quantile discretisation
print("Edges of the discrtisation cut: ", bins_edges)
#Documentation
#https://towardsdatascience.com/how-i-customarily-bin-data-with-pandas-9303c9e4d946#005a

# Suppression  of quantitative variables for MCA
#----------------------------------------------------
dashMCA=dashMCA.drop(["Marks"],axis=1)

#Creates Indicators
#----------------------------------------------------
#dc=pd.DataFrame(pd.get_dummies(dashMCA[["progress","module","type","MarksQ"]]))

In [ ]:
dashMCA.head()

Python libraries are poor in graphic functions directly adapted to the MCA. We will therefore save the table and continue the analysis with R

In [ ]:
#dashMCA.to_csv('dashMCA.csv')

-----------------------------------------------------------------------------
# Draft and unfinished codes
-----------------------------------------------------------------------------

__TO DO LIST__
- préprocesser la durée : il faut enlever le minutes -> FAIT
- student et supervisor : nombre et dispersion -> FAIT
- année : nombre; date max et date min -> FAIT
- attention : bcp d'élèves avec pas de mark -> OK
- regarder pour chaque prof et élève la date de ses meetings et la durée de ceux-ci si possible
- note moyenne selon l'année et le tuteur -> FAIT
- racé de la note moyenne donnée par le superviseur -> FAIT
- note moyenne selon la matière -> FAIT
- MCA sur les variables qualitatives  avec imputation -> FAIT 
- MCA sur les variables qualitatives  avec suppression des données 

bcp de données qualitatives et peu de données quantitatives (définir lesquelles proprement dans un markdown)
algos possibles : MCA (p var quali),LDA (p var quali et 1 quanti), boxplots et graphes
- appréciation majoritaire donnée par la prof 
- relation entre appréciation moyenne et note obtenue 
- faire du clustering sur les élèves et leur note? 

- tester analyse de variance univariée/multivariée

The main distinction between the two methods is:
- loc: gets rows (and/or columns) with particular labels.
- iloc: gets rows (and/or columns) at integer locations.
- pandas.Series: = dataframe column

In [ ]:
####### VERSION 2 #######
ss = ["10","10min","/","20 min","140","0.5h","1hour","1H","1 hour", "1 Hour","h1.5","H 0.5","1+5+7+7"]
result = np.zeros(len(ss))
i = 0
for s in ss:
   
    #extraction of integer from string
    rr = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", s)
    #rr = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?\s*h*H*", s)
    #rr = re.findall("[hH]*\s*[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?\s*[hH]*", s)
    
    if len(rr) != 0: 
        rr = float(rr[0]) #transform string: "1.3" to float: 1.3
        #if there is an h or H in the string it means that the time is in hours and not in minutes
        rh = re.findall("[hH]+",s)
        #in that case convert hours to minutes
        if len(rh) != 0:
            rr = rr*60
    else: #put nan if unrecognised character
        rr = np.nan 
    result[i] = rr
      
    i += 1
    
    print("s: ",s)
    print("rr: ",rr)
    print("---------")
    #print("rh: ",rh)
    

#documentation:    
#https://docs.python.org/3/library/re.html
#https://stackoverflow.com/questions/4289331/how-to-extract-numbers-from-a-string-in-python/4289415#4289415
#https://dl.icewarp.com/online_help/203030104.htm

In [ ]:
###### INVERSE MATCHING ########
ss = ["10","10min","/","20 min","140","0.5h","1hour","1H","1 hour", "1 Hour","h1.5","H 0.5","1+5+7+7","Andrea","5x2", "13:30-14:30"]
for s in ss:
    
    #ru = re.findall("^(?:(?![minutehoursMINUTEHOURs]).)*",s)
    #ru = re.findall("^(?:(?![minutehoursMINUTEHOURs\d]).)*",s)
    ru = re.findall("(?:(?![minutehoursMINUTEHOURs\f\d.,\s*]).)*",s)  #^(.(?!(some text)))*$
    
    for elem in ru:
        if len(elem) != 0: #if there is an unknown caracter
            ru = np.nan
            break

    print("s: ",s)
    print("ru: ",ru)
    print("-------")

In [ ]:
####### VERSION 3 #######

ss = ["10","10min","/","20 min","140","0.5h","1hour","1H","1 hour", "1 Hour","h1.5","H 0.5","1+5+7+7","Andrea","5x2", "13:30-14:30"]
result = np.zeros(len(ss))
i = 0

for s in ss:
    #extraction of any unknown caracter from the string i.e. find everything exept:
    #characters: minutehoursMINUTESHOURS, integer, floats, blank space, . 
    ru = re.findall("(?:(?![minutehoursMINUTEHOURs\f\d.,\s*]).)*",s)
    unknown = 0
    
    for elem in ru: #check if there is an unknown caracter in the string
        if len(elem) != 0: 
            unknown = 1
            break  
    print("unknown: ",unknown)
            
    if unknown == 1: #if there is an unknown caracter in the string, put the string to nan 
        result[i] = np.nan
    else:  #if not, extraction of the time from the string
        #extraction of integers regardless of their position from the string
        rr = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", s)

        if len(rr) != 0: #if there is a time in the string
            rr = float(rr[0]) #transform string: "1.3" to float: 1.3
            #if there is an h or H in the string it means that the time is in hours and not in minutes
            rh = re.findall("[hH]+",s)
            #in that case convert hours to minutes
            if len(rh) != 0:
                rr = rr*60
        result[i] = rr
    
    print("s: ",s)
    print("result: ",result[i])
    print("---------")
    i += 1
    #print("rh: ",rh)

In [ ]:
fig,ax=plt.subplots(figsize=(18, 6))#creating subplots
width = 0.2
idx=np.sort(dfProg["Supervisor_ID"].values)
ax.bar(x=idx, height='satisfactory', width= width, align='center', data=dfProg,label = 'satisfactory')
ax.bar(x=idx+width, height='moderate', width= width, align='center', data=dfProg,label = 'moderate')
ax.bar(x=idx+width*2, height='unsatisfactory', width= width, align='center', data=dfProg,label = 'unsatisfactory')
ax.set_xticks(idx)
ax.set_xlabel('Supervisor ID')
ax.set_ylabel('Progress')
#ax.legend()
fig.tight_layout()
plt.show()

In [ ]:
#---progress per Supervisor
ax3=plt.subplots(figsize=(15, 5))#creating subplots
ax3=sns.barplot(x = "Supervisor_ID",y = 'satisfactory', data = dfProg,
            label = 'Satisfactory', color = 'g', edgecolor = 'w')
ax3=sns.barplot(x = "Supervisor_ID",y = 'moderate', data = dfProg,
            label = 'Moderate', color = 'mediumblue', edgecolor = 'w')
ax3=sns.barplot(x = "Supervisor_ID",y = 'unsatisfactory', data = dfProg,
            label = 'Unsatisfactory', color = 'red', edgecolor = 'w')
ax3.set(xlabel="Supervisor ID", ylabel="Progress")
ax3.legend()
ax3.set_title('Number of progress "satisfactory", "moderate", "unsatisfactory" for each Supervisor ')
sns.despine(left = True, bottom = True)  #remove graph frame
plt.show()

In [ ]:
studID = 5 #17
idx = np.where(dashbord["Student"]==studID) #index (no row names i.e. use .iloc[idx]) associated with student 
stuMark = dashbord["Marks"].iloc[idx[0][0]].astype(int)
#do not use idx after since it is not sorted by time. Use the dates.index instead

dates = dashbord["date"].iloc[idx].sort_values()#sort the dates by ascending order

#These tabs values are in the same order as the dates (ascending order):
meetTab0 = np.zeros(len(dates),dtype=int) #duration of the meeting 
meetTab1 = np.zeros(len(dates),dtype=str) #progress of the meeting 
meetTab2 = np.zeros(len(dates),dtype=str) #type of the meeting: consider only approval or meeting and no ethics 

#np.nan == np.nan returns False. We have to define a nan for the strings
nan_str = np.array([np.nan]).astype(str)[0]


print("-----------------Original data sorted by date")
print(dates); print(len(dates)); print()
print(dashbord["duration"].loc[dates.index]); print(len(dates)); print()
print(dashbord["progress"].loc[dates.index]); print(len(dates)); print()
print(dashbord["type"].loc[dates.index]); print(len(dates)); print()

print("-----------------data in meetTab")
meetTab0 = dashbord["duration"].loc[dates.index].to_numpy() #ok you have to use .loc[]
meetTab1 = dashbord["progress"].loc[dates.index].to_numpy()
meetTab2 = dashbord["type"].loc[dates.index].to_numpy()

nbMeetin = len(np.where(meetTab2=='meeting')[0])
nbApprov = len(np.where(meetTab2=='approval')[0])
nbEthics = len(np.where(meetTab2=='ethics')[0])

print("meetTab0:", meetTab0);print(len(meetTab0)); print()
print("meetTab1:", meetTab1);print(len(meetTab1)); print()
print("meetTab2:", meetTab2);print(len(meetTab2)); print()

print("-----------------")

idxEthics = np.where(meetTab2=='ethics')#gets the index where there is the type 'ethics'.
print("idxEthics:", idxEthics)

#--- delete the column corresponding to 'ethics' in the 3 tabs and in the dates
meetTab0 = np.delete(meetTab0,idxEthics)
meetTab1 = np.delete(meetTab1,idxEthics)
meetTab2 = np.delete(meetTab2,idxEthics) 
dates = dates.drop(labels=dates.index[idxEthics])

print("-----------------removes data from meetTab")
print(dates); print(len(dates)); print()
print("meetTab0:", meetTab0); print();print(len(meetTab0)); print()
print("meetTab1:",meetTab1); print();print(len(meetTab1)); print()
print("meetTab2:",meetTab2); print();print(len(meetTab2)); print()

nanTab1 = np.where(meetTab1.astype(str)==nan_str)
meetTab1[nanTab1]='unkown' #'unkown' if no progress found otherwise the 3 np.where below don't work
print("meetTab1 after removing nan",meetTab1)
#see: https://stackoverflow.com/questions/46055853/cant-find-nan-entries-using-numpy-in-array-of-strings

nanTab2 = np.where(meetTab2.astype(str)==nan_str)
meetTab2[nanTab2]='unkown'

#--- transfrom meetTab1 elements in float values to be able to plot it.
Satif = np.where(meetTab1=='satisfactory')
Moder = np.where(meetTab1=='moderate')
Unsat = np.where(meetTab1=='unsatisfactory')

#if the duration array is not empty, change de meetTab0 value so it can be plotted 
#on the same graph as the duration
if not np.all(np.isnan(meetTab0)):
    meetTab1[Satif] = np.nanmax(meetTab0) #nanmax ignore nan values
    meetTab1[Moder] = (np.nanmax(meetTab0)+np.nanmin(meetTab0))/2
    meetTab1[Unsat] = np.nanmin(meetTab0)
    meetTab1[nanTab1] = np.nan #the value is reset to nan
else: #otherwise, put a fixed value
    meetTab1[Satif] = 3
    meetTab1[Moder] = 2
    meetTab1[Unsat] = 1
    meetTab1[nanTab1] = np.nan #the value is reset to nan

#---transforms 'approval' and 'meeting' values into M and A for better readability when plotting
meetTab2[np.where(meetTab2=='approval')] = 'A'
meetTab2[np.where(meetTab2=='meeting')] = 'M'

#--- associate a color in the barplot
barColors = np.full(len(meetTab1),'w') #to plot the color of the barplot
barColors[Satif] = 'g'
barColors[Moder] = 'b'
barColors[Unsat] = 'r'

#print(dates.index)

In [ ]:
nbApprov

In [ ]:
x_axis = np.arange(len(dates))

f, ax = plt.subplots(figsize=(8, 5),sharex=True)
plt.bar(x=x_axis, height=meetTab1, color = 'g',width=0.5, align='center',edgecolor = 'w',alpha=0.5,zorder=1)
plt.scatter(x=x_axis,y=meetTab0,label='duration',edgecolor = 'w', zorder=2)

ax.xaxis.set_ticks(x_axis)
ax.xaxis.set_ticklabels(dates) # change the ticks' names to x
plt.xticks(rotation = 90)

"""dfmt = DateFormatter("%Y-%m-%d") # proper formatting Year-month-day
ax.xaxis.set_major_formatter(dfmt)"""

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

#background color
ax.patch.set_facecolor('grey')
ax.patch.set_alpha(0.05)


f.tight_layout()

In [ ]:
#méth 2
dfDate = pd.DataFrame(datesU)
dfDate = dfDate.rename(columns={0: "date"}) 
dfDate["date"] = pd.to_datetime(dfDate["date"])
dg = dfDate.groupby(pd.Grouper(key='date', freq='1M')).sum() # groupby each 1 month
dg.index = dg.index.strftime('%B')
dg

In [ ]:
startDate = df.date.min(); print(startDate)
endDate = df.date.max(); print(endDate)

#creation of a date vector between 2 input dates
vectD = pd.date_range(startDate.date(),endDate.date()-timedelta(days=1),freq='M'); print(vectD)

"""startDate = datetime.strptime(startDate, '%Y-%m-%d %H:%M:%S') #convert to datetime
endDate = datetime.strptime(endDate, '%Y-%m-%d %H:%M:%S')
"""
"""strDate = '2018-07-11 00:00:00'
endDate = datetime.strptime(strDate, '%Y-%m-%d %H:%M:%S'); print(endDate) #convert string to type date"""

Idate1 = 0
Idate2 = 5

"""idxStartDate = df[df.eq(startDate).any(1)].index[0]; print(idxStartDate)#row index i.e. use .loc[]
idxEndDate = df[df.eq(endDate).any(1)].index[0]; print(idxEndDate) #row index i.e. use .loc[]
newdf = df.loc[idxStartDate:idxEndDate]"""

#create a DataFrale with the dates between 2 values vectD[Idate1] and vectD[Idate2]m
newdf = df[df['date'].between(vectD[Idate1], vectD[Idate2])]

irina: research assistant. Work for epics
3 research assistant andrew : ML area of expertise (time series, analysis satistics, data analysis but involed in many things but mostly data analysis) has been in sport science, computer software
annalina : research assistant

In [ ]:
#Computation of the mean mark per supervisor
#-------------------------------------------------
def computeMeanMark(supIDG,stuIndex):
    #stuIndex = Indices (row labels in the dataframe i.e. use .loc[stuIndex]) 
    #of students who received a grade
    #supIDG = among students who received a grade, ID of supervisors who are not NaN values. 
    meanMark = np.zeros((2,len(supIDG))) #mean mark per Supervisor. Row 0: supervisor ID, Row 1: mark
    meanMark[0,:] = supIDG #Row 0: supervisor ID
    i = 0
    for sup in supIDG:
        idx = np.where(dashbord["Supervisor"].loc[stuIndex] == sup)
        meanMark[1,i] = mean(dashbord["Marks"].loc[stuIndex].iloc[idx])
        i += 1    

    #compute the number of students per supervisor
    #-----------------------------------------------------
    nbStudperSup = [] #nb of student per supervisor in the same order as the supervisors in the dataframe
    supList = dashbord["Supervisor"].drop_duplicates().values #list of all supervisors

    for s in supList:
        idxS = np.where(dashbord["Supervisor"]==s) #indexes (use .iloc[]) where the supervisor is s
        nbStudperSup.append(len(dashbord["Student"].iloc[idxS].drop_duplicates().values))

    #Data fromatting
    #-------------------------------------------------
    dfmeanMark = pd.DataFrame(meanMark.T) #transformation to dataframe to plot graph with seaborn
    dfmeanMark = dfmeanMark.rename(columns={0: "Supervisor_ID", 1: "Mean_mark"})
    dfmeanMark['Supervisor_ID'] = dfmeanMark['Supervisor_ID'].astype(int)
    dfmeanMark["nb_students"] = pd.DataFrame(nbStudperSup)
    
    return(dfmeanMark)

In [ ]:
a = dashbord[["Marks","Student","Supervisor"]].drop_duplicates()
a = a.sort_values(by=['Supervisor'])
supList = a["Supervisor"].drop_duplicates()#list of supervisor IDs
supList = np.delete(supList.values,np.where(supList.isna() == True)) #delete NaN value from supList 
meanMark = np.zeros(len(supList)) #mean Mark for each supervisor (in the same order as supList)
nbStudperSup = np.zeros(len(supList)) #number of students per Supervisor
i=0
for s in supList:
    meanMark[i] = mean(a["Marks"].iloc[np.where(a["Supervisor"]==s)]) #mean skip NaN values unless there is only Nan
    students = a["Student"].iloc[np.where(a["Supervisor"]==s)].values #student IDs associated with supervisor s
    students = np.delete(students,np.where(np.isnan(students)==True)) #delete student value equals to NaN 
    nbStudperSup[i] = len(students) #number of students for supervisor s 
    i+=1

In [ ]:
#computation of the progress given by each Supervisor
#-------------------------------------------------
################"" SAUVEGARDE ANcienne version ## 
def computeProgressSup(name = "Supervisor",stacked = False):
    supID = dashbord[name].drop_duplicates() #ID of all supervisors even those who did not give a mark (!= supIDG)
    supID = delete(supID.values,np.where(supID.isna() == True)) #delete value NaN from supID
    Prog = np.zeros((3,len(supID))) #tab containing the progress for each supervisor
    #columns : supervisors (in the same order as supID)
    #row 0: number of 'satisfactory' progress
    #row 1: number of 'moderate' progress
    #row 2: number of 'unsatisfactory' progress
    i=0
    for s in supID:
        idx = np.where(dashbord[name]==s) #index (raw index no row name) of the location of supervisor s

        #--- for 'satisfactory'
        idx_satis = np.where(dashbord["progress"].iloc[idx]=="satisfactory") #index of 'satisfactory' progress  
        nb_satis = len(dashbord["progress"].iloc[idx].iloc[idx_satis]) #number of 'satisfactory' for supervisor s
        Prog[0,i] = nb_satis
        #--- for 'moderate'
        idx_moder = np.where(dashbord["progress"].iloc[idx]=="moderate") #index of 'moderate' progress  
        nb_moder = len(dashbord["progress"].iloc[idx].iloc[idx_moder]) #number of 'moderate' for supervisor s
        Prog[1,i] = nb_moder
        #--- for 'unsatisfactory'
        idx_unsat = np.where(dashbord["progress"].iloc[idx]=="unsatisfactory") #index of 'unsatisfactory' progress  
        nb_unsat = len(dashbord["progress"].iloc[idx].iloc[idx_unsat]) #number of 'unsatisfactory' for supervisor s
        Prog[2,i] = nb_unsat

        #divide by the total number of 'progress' given by the supervisor 
        Sum = nb_satis+nb_moder+nb_unsat
        if Sum != 0:
            Prog[:,i] = Prog[:,i]/(Sum)
        else:
            Prog[:,i] = 0
        i+=1

    #Data formatting
    #-------------------------------------------------
    dfProg = pd.DataFrame(Prog.T) #transformation to dataframe to plot graph with seaborn
    dfProg = dfProg.rename(columns={0: "satisfactory", 1: "moderate", 2: "unsatisfactory"})      
    dfProg[name+'_ID'] = supID.astype(int)  
    
    #Data formatting for stacked histogram
    #-------------------------------------------------
    if stacked == True:
        dfProg.index = dfProg[name+'_ID']
        dfProg = dfProg[["unsatisfactory", "moderate", "satisfactory"]]
    return(dfProg)

In [ ]:
#Plot of the graphs
#-------------------------------------------------
########################### VERSION 1: Non Stacked Histogram ########################### 
f, (ax1,ax3) = plt.subplots(2,1,figsize=(15, 10))
sns.set_color_codes('pastel')

#---progress per Supervisor
sns.barplot(ax = ax1, x = "Supervisor_ID",y = 'satisfactory', data = dfProg,
            label = 'proportion of satisfactory', color = 'g', edgecolor = 'w')
sns.barplot(ax = ax1, x = "Supervisor_ID",y = 'moderate', data = dfProg,
            label = 'proportion of moderate', color = 'mediumblue', edgecolor = 'w')
sns.barplot(ax = ax1, x = "Supervisor_ID",y = 'unsatisfactory', data = dfProg,
            label = 'proportion of unsatisfactory', color = 'red', edgecolor = 'w')
ax1.set(xlabel="Supervisor ID", ylabel="Progress")

#--- add mean mark per Supervisor
ax2 = ax1.twinx()
#so that there is no shift between the graphs ax1 and ax2
xvalues = np.sort(dfmeanMark["Supervisor_ID"].values) - 1 
sns.scatterplot(ax = ax2,x = xvalues, y = 'Mean_mark', data = dfmeanMark,color = 'slateblue',edgecolor = 'w')
sns.lineplot(ax = ax2,x = xvalues, y = 'Mean_mark', data = dfmeanMark,alpha = 0.5,
             label = 'mean mark', color = 'slateblue')
ax2.set(ylabel="mean mark")

ax1.set_title('Progress and mean mark for each Supervisor')
sns.despine(left = True, bottom = True)  #remove graph frame
ax1.legend(bbox_to_anchor= (1.1, 1.2))
ax2.legend(bbox_to_anchor= (1.05, 1.3))
plt.grid(linestyle = 'dotted')

#---mark dispersion
flierprops = dict(markerfacecolor='0.75', markersize=4,linestyle='none') #size & color of outliers
sns.boxplot(ax = ax3,x='Supervisor', y='Marks', data=dashbord,flierprops=flierprops,linewidth=1)
xlabels = np.sort(dfmeanMark["Supervisor_ID"].values)
ax3.set_xticklabels(ax3.get_xticks().astype(int)) #force integer xlabels
ax3.set(ylabel="mark dispersion", xlabel="Supervisor ID")
ax3.set_title('Mark dispersion and number of students for each Supervisor')

#--- add number of students per Supervisor
ax4 = ax3.twinx()
sns.scatterplot(ax = ax4, x = xvalues, y = 'nb_students', data = dfmeanMark,color = 'red',edgecolor = 'w')
sns.lineplot(ax = ax4, x = xvalues, y = 'nb_students', data = dfmeanMark,alpha = 0.5,
             label = 'number of students', color = 'red')
ax4.set(ylabel="number of students")
ax4.legend(bbox_to_anchor= (1.05, 1.2))

plt.subplots_adjust(hspace=0.4) #set space between the 2 plots
f, (ax1,ax3) = plt.subplots(2,1,figsize=(15, 10))
sns.set_color_codes('pastel')

#---progress per Supervisor
sns.barplot(ax = ax1, x = "Supervisor_ID",y = 'satisfactory', data = dfProg,
            label = 'proportion of satisfactory', color = 'g', edgecolor = 'w')
sns.barplot(ax = ax1, x = "Supervisor_ID",y = 'moderate', data = dfProg,
            label = 'proportion of moderate', color = 'mediumblue', edgecolor = 'w')
sns.barplot(ax = ax1, x = "Supervisor_ID",y = 'unsatisfactory', data = dfProg,
            label = 'proportion of unsatisfactory', color = 'red', edgecolor = 'w')
ax1.set(xlabel="Supervisor ID", ylabel="Progress")

#--- add mean mark per Supervisor
ax2 = ax1.twinx()
#so that there is no shift between the graphs ax1 and ax2
xvalues = np.sort(dfmeanMark["Supervisor_ID"].values) - 1 
sns.scatterplot(ax = ax2,x = xvalues, y = 'Mean_mark', data = dfmeanMark,color = 'slateblue',edgecolor = 'w')
sns.lineplot(ax = ax2,x = xvalues, y = 'Mean_mark', data = dfmeanMark,alpha = 0.5,
             label = 'mean mark', color = 'slateblue')
ax2.set(ylabel="mean mark")

ax1.set_title('Progress and mean mark for each Supervisor')
sns.despine(left = True, bottom = True)  #remove graph frame
ax1.legend(bbox_to_anchor= (1.1, 1.2))
ax2.legend(bbox_to_anchor= (1.05, 1.3))
plt.grid(linestyle = 'dotted')

#---mark dispersion
flierprops = dict(markerfacecolor='0.75', markersize=4,linestyle='none') #size & color of outliers
sns.boxplot(ax = ax3,x='Supervisor', y='Marks', data=dashbord,flierprops=flierprops,linewidth=1)
xlabels = np.sort(dfmeanMark["Supervisor_ID"].values)
ax3.set_xticklabels(ax3.get_xticks().astype(int)) #force integer xlabels
ax3.set(ylabel="mark dispersion", xlabel="Supervisor ID")
ax3.set_title('Mark dispersion and number of students for each Supervisor')

#--- add number of students per Supervisor
ax4 = ax3.twinx()
sns.scatterplot(ax = ax4, x = xvalues, y = 'nb_students', data = dfmeanMark,color = 'red',edgecolor = 'w')
sns.lineplot(ax = ax4, x = xvalues, y = 'nb_students', data = dfmeanMark,alpha = 0.5,
             label = 'number of students', color = 'red')
ax4.set(ylabel="number of students")
ax4.legend(bbox_to_anchor= (1.05, 1.2))

plt.subplots_adjust(hspace=0.4) #set space between the 2 plots
sns.despine(left = True, bottom = True) #remove graph framesns.despine(left = True, bottom = True) #remove graph frame

In [ ]:
########################### VERSION 1: Non Stacked Histogram ########################### 
class StudentProgress_Dashbord(param.Parameterized):
    Min = int(dashbord["Student"].min())
    Max = int(dashbord["Student"].max())
    Select_begin = param.Integer(Min,bounds=(Min,Max))
    Select_end = param.Integer(70,bounds=(Min,Max))    
    def view(self):
        begin = self.Select_begin
        end = self.Select_end
        xvalue,yvalueS,yvalueM,yvalueU,yMarks = data_StudentProgress()
        #---progress per Student
        fontS = 6
        fontL = 8
        f, ax= plt.subplots(figsize=(7, 3),constrained_layout=True)
        sns.barplot(ax= ax, x = xvalue[begin:end],y =yvalueS[begin:end],
                label = 'proportion of satisfactory', color = 'g', edgecolor = 'w')
        sns.barplot(ax = ax, x =  xvalue[begin:end],y = yvalueM[begin:end], 
                label = 'proportion of moderate', color = 'mediumblue', edgecolor = 'w')
        sns.barplot(ax = ax, x =  xvalue[begin:end],y = yvalueU[begin:end],
                label = 'proportion of unsatisfactory', color = 'red', edgecolor = 'w')
        ax.set_xlabel("Student ID",fontsize=fontS)
        ax.set_ylabel("Progress",fontsize=fontS)
        ax.tick_params(axis='x', which='major',rotation=90)
        ax.tick_params(axis='both', which='major', labelsize=fontS)

        #--- add student mark
        ax2 = ax.twinx()
        #so that there is no shift between the graphs ax1 and ax2
        ax2_xval = np.arange(0,len(yMarks[begin:end]),1)
        sns.scatterplot(ax = ax2,x = ax2_xval, y = yMarks[begin:end], color = 'slateblue',
                        edgecolor = 'w',s = 15)
        sns.lineplot(ax = ax2,x = ax2_xval, y = yMarks[begin:end], alpha = 0.5, linewidth  = 1,
                 label = 'student mark', color = 'slateblue')
        ax2.set_ylabel("mean mark",fontsize=fontS)
        ax2.tick_params(axis='both', which='major', labelsize=fontS)

        ax.set_title('Progress and mark for each Student',fontsize=fontL)
        sns.despine(left = True, bottom = True)  #remove graph frame
        ax.legend(bbox_to_anchor= (1.1, 1.25),fontsize=fontS)
        ax2.legend(bbox_to_anchor= (1.05, 1.35),fontsize=fontS)
        plt.grid(linestyle = 'dotted')
        plt.close(f)
        return(f)

In [ ]:
df["unsatisfactory_box"] = np.zeros(len(df)) #add column to know if the students fells witihin the unsatisfactory box 
df["moderate_box"] = np.zeros(len(df)) #add column to know if the students fells witihin the moderate box 
df["satisfactory_box"] = np.zeros(len(df)) #add column to know if the students fells witihin the satisfactory box 
stuList = df["Student"].dropna().drop_duplicates().values
for s in stuList:
    idx = np.where(df["Student"]==s) #indexes where Student ID = s
    length_s = len(df.iloc[idx])

    IsUnsat = Counter(df["progress"].iloc[idx].values)["unsatisfactory"] >= 1 #True if there is at least 1 unsatisfactory progress
    IsModer = not(IsUnsat) and Counter(df["progress"].iloc[idx].values)["moderate"] >= 1 #True if there is no unsatisfactory progress but at least 1 moderate progress 
    IsSatif = not(IsUnsat) and not (IsModer) and Counter(df["progress"].iloc[idx].values)["satisfactory"] >= 1 #True if there is only satisfactory progress 

    if IsUnsat: 
        df["unsatisfactory_box"].iloc[idx] = np.ones(len(df.iloc[idx]))    
    elif IsModer: 
        df["moderate_box"].iloc[idx] = np.ones(len(df.iloc[idx]))
    elif IsSatif: 
        df["satisfactory_box"].iloc[idx] = np.ones(len(df.iloc[idx]))

dfUnsat = df.iloc[np.where(df["unsatisfactory_box"]==1)]#create new dataframe with only students who fell into the unsatisfactory box
dfModer = df.iloc[np.where(df["moderate_box"]==1)]#create new dataframe with only students who fell into the moderate box
dfSatis = df.iloc[np.where(df["satisfactory_box"]==1)]#create new dataframe with only students who fell into the satisfactory box